# FSKU 데이터 증강 및 RAG 시스템 (통합본)
## 답변 생성 보장 + 통합형/분리형 선택 가능

### 🎯 주요 특징
1. **통합형 생성**: 질문과 답변을 한 번에 생성 (빠름, 일관성)
2. **분리형 생성**: 질문 생성 후 답변 생성 (정확함, 유연함)
3. **답변 보장**: 3단계 폴백 시스템으로 답변 생성 보장
4. **RAG 통합**: 문서 기반 정확한 답변 생성
5. **실험 가능한 설정**: 모든 파라미터 조정 가능

# 🗺️ 코드 실행 흐름 가이드 (백엔드 개발자를 위한)

## 🎯 이 노트북의 목적
금융 AI 모델을 학습시키기 위한 **고품질 데이터를 자동으로 생성**하는 파이프라인 구축

## 📋 실행 순서와 각 단계의 역할

### 1️⃣ **환경 설정** (Setup)
```
백엔드 비유: 서버 초기화 및 의존성 주입
```
- GPU 설정 = 서버 리소스 할당
- 라이브러리 임포트 = npm install / pip install
- 디렉토리 구조 = 프로젝트 스캐폴딩

### 2️⃣ **설정 클래스** (Configuration) 
```
백엔드 비유: application.yml / config.json
```
- ExperimentConfig = 환경변수 관리
- 하이퍼파라미터 = API rate limit, timeout 설정
- 프리셋 = dev/staging/prod 환경 설정

### 3️⃣ **RAG 시스템** (Knowledge Base)
```
백엔드 비유: 외부 DB 연결 및 캐싱 레이어
```
- PDF 로드 = 데이터베이스 연결
- 벡터 인덱싱 = Elasticsearch 인덱싱  
- 유사도 검색 = Full-text search
- 캐싱 = Redis 캐싱

### 4️⃣ **프롬프트 템플릿** (Templates)
```
백엔드 비유: API 요청/응답 스키마 정의
```
- 프롬프트 = Request DTO
- 생성된 텍스트 = Response DTO
- 파싱 = Serialization/Deserialization

### 5️⃣ **데이터 생성기** (Generator)
```
백엔드 비유: 비즈니스 로직 레이어
```
- 모델 로드 = 서비스 초기화
- generate_text() = 핵심 비즈니스 로직
- 통합형/분리형/CoT = 다른 알고리즘 전략 패턴

### 6️⃣ **품질 검증** (Validation)
```
백엔드 비유: 유효성 검사 및 에러 핸들링
```
- 품질 점수 = Validation rules
- 재시도 로직 = Retry mechanism
- 폴백 = Circuit breaker pattern

### 7️⃣ **배치 처리** (Batch Processing)
```
백엔드 비유: 배치 잡 실행
```
- 대량 생성 = Batch job
- 진행상황 추적 = Job monitoring
- 중간 저장 = Checkpointing

## 💡 핵심 실행 경로

```python
# 메인 실행 흐름
1. config = ExperimentConfig()          # 설정 로드
2. rag = RAGSystem(config)              # RAG 초기화
3. rag.initialize()                     # 인덱스 구축/로드
4. generator = AnswerGuaranteedGenerator(config, rag)  # 생성기 생성
5. generator.load_model()               # 모델 로드
6. for context in contexts:
      qa_pair = generator.generate_qa_pair(context)  # 데이터 생성
7. save_results(qa_pairs)              # 결과 저장
```

## 🔄 각 모드별 내부 흐름

### 통합형 (Integrated)
```
Context → LLM → Q&A 동시 생성 → 파싱 → 검증 → 저장
```

### 분리형 (Separated) - 더 정확\!
```
Context → LLM → 질문 생성
    ↓
질문으로 RAG 재검색
    ↓
Context + RAG 결과 → LLM → 답변 생성
    ↓
검증 → 저장
```

### CoT (Chain-of-Thought) - 최고 품질\!
```
초기 생성 → 자가 검증 → 개선 → 최종 검증
   ↓           ↓           ↓         ↓
  70점?      문제 발견    수정      85점?
```

## 🎮 실전 사용법

```python
# 1. 빠른 테스트 (5분)
config.GENERATION_MODE = "integrated"
config.BATCH_CONFIG['target_count'] = 10

# 2. 품질 우선 (30분)
config.GENERATION_MODE = "cot"
config.COT_CONFIG['use_cot'] = True
config.CURRENT_COT_PRESET = "quality"

# 3. 대회 제출용 (2시간)
config.GENERATION_MODE = "separated"
config.BATCH_CONFIG['target_count'] = 1000
```

# 🎓 백엔드 개발자를 위한 AI 핵심 개념

## 🤖 LLM (Large Language Model)
```
백엔드 비유: 초거대 함수
- 입력: 텍스트 (Request)
- 처리: 수십억 개 파라미터로 계산
- 출력: 텍스트 (Response)
```

### 주요 개념 매핑
| AI 용어 | 백엔드 용어 | 설명 |
|---------|------------|------|
| Model | Service/Engine | 실제 처리를 담당하는 핵심 컴포넌트 |
| Tokenizer | Parser/Serializer | 텍스트 ↔ 숫자 변환 |
| Inference | API Call | 모델에 요청 보내고 응답 받기 |
| Fine-tuning | Customization | 특정 도메인에 맞게 커스터마이징 |
| Prompt | Request Body | 모델에 보내는 입력 |
| Temperature | Randomness Config | 응답의 다양성 조절 (0=결정적, 1=창의적) |
| Batch Size | Connection Pool Size | 동시 처리 개수 |
| Learning Rate | Update Speed | 학습 속도 (너무 빠르면 불안정) |

## 🧮 양자화 (Quantization)
```python
# 백엔드 비유: 데이터 압축
# 원본: {"price": 123.456789} (float64)
# 압축: {"price": 123.46} (float16)
# 더 압축: {"price": 123} (int8)

# AI에서의 양자화
원본 모델: 28GB (FP32)
↓ 양자화
압축 모델: 3.5GB (INT4)
# 메모리 87.5% 절약\!
```

## 🔗 LoRA (Low-Rank Adaptation)
```python
# 백엔드 비유: 어댑터 패턴

class OriginalService:  # 거대한 원본 모델 (수정 불가)
    def process(self, input):
        return expensive_computation(input)

class LoRAAdapter:  # 작은 어댑터 (학습 가능)
    def __init__(self, rank=16):  # rank = 어댑터 크기
        self.adapter_weights = small_matrix(rank)
    
    def process(self, input):
        original_output = OriginalService.process(input)
        adapter_output = self.adapter_weights @ input
        return original_output + adapter_output  # 원본 + 어댑터

# 장점: 원본은 그대로, 어댑터만 학습 → 메모리 99% 절약\!
```

## 📚 RAG (Retrieval-Augmented Generation)
```python
# 백엔드 비유: 캐시 + 외부 API 패턴

def generate_answer(question):
    # 1. 캐시(DB) 검색
    relevant_docs = search_database(question)  # SELECT * FROM docs WHERE ...
    
    # 2. 컨텍스트 보강
    context = f"{question}\n관련 정보: {relevant_docs}"
    
    # 3. LLM 호출
    answer = llm.generate(context)  # 외부 API 호출처럼
    
    return answer

# RAG 없이: "바젤III가 뭐야?" → LLM이 학습한 내용만으로 답변
# RAG 있으면: "바젤III가 뭐야?" → DB 검색 → 최신 정보 포함해서 답변
```

## 🔄 Fine-tuning 프로세스
```python
# 백엔드 비유: 점진적 배포 (Canary Deployment)

# 1. Pre-trained Model (기본 모델)
base_model = load_model("gpt-base")  # npm install express

# 2. Add Custom Layer (커스텀 레이어)
custom_layer = LoRAAdapter()  # 우리 비즈니스 로직

# 3. Training Loop (학습 루프)
for epoch in range(3):  # 3번 반복
    for batch in training_data:  # 배치 단위 처리
        loss = compute_loss(batch)  # 에러 계산
        update_weights(loss)  # 가중치 업데이트
        
        # 백엔드의 모니터링처럼
        if step % 100 == 0:
            log_metrics(loss, accuracy)
            save_checkpoint()  # 중간 저장
```

## 💾 메모리 관리 전략
```python
# RTX 4090 (24GB) 기준

# ❌ 나쁜 예: OOM (Out of Memory)
model = load_model("70B-model")  # 70B = 280GB 필요\!

# ✅ 좋은 예: 메모리 최적화
model = load_model("7B-model", quantization="4bit")  # 3.5GB만 사용
optimizer = "paged_adamw"  # 페이징으로 메모리 절약
gradient_checkpointing = True  # 메모리 ↔ 속도 트레이드오프

# 백엔드 비유: 
# - Quantization = Response 압축 (gzip)
# - Gradient Checkpointing = Lazy Loading
# - Paged Optimizer = Swap 메모리 활용
```

## 🎯 하이퍼파라미터 튜닝 가이드
```python
# 백엔드의 성능 튜닝과 유사

# 1. Learning Rate (처리 속도)
# - 너무 높음 = 429 Too Many Requests (발산)
# - 너무 낮음 = 408 Request Timeout (학습 안됨)
# - 적절함 = 200 OK

# 2. Batch Size (동시 요청 수)
# - 너무 큼 = 503 Service Unavailable (OOM)
# - 너무 작음 = 비효율적 (느림)
# - 적절함 = Thread Pool Size처럼 조절

# 3. Temperature (응답 다양성)
# - 0.1 = Deterministic (항상 같은 응답)
# - 0.7 = Balanced (적당한 변화)
# - 1.5 = Creative (예측 불가능)
```

In [ ]:
# ========================================
# 🎓 AI 개발 실습: 환경 설정 및 라이브러리 임포트
# ========================================
# 
# 💡 이 섹션에서 배우게 될 내용:
#   1. AI 개발에 필요한 핵심 라이브러리들의 역할
#   2. GPU 메모리 관리 방법
#   3. 프로젝트 구조 설정 방법

import os
import json
import time
import pickle
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd
from tqdm import tqdm  # 진행 상황을 보여주는 프로그레스 바 라이브러리

# 🔍 딥러닝 프레임워크 임포트
import torch  # PyTorch: 딥러닝의 핵심 프레임워크
import torch.nn as nn  # 신경망 모듈들 (실제로는 Transformer 모델이 이미 구현되어 있어 직접 사용할 일은 적음)

# 🤗 Hugging Face 라이브러리들 - LLM 사용의 핵심
from transformers import (
    AutoModelForCausalLM,     # 자동으로 모델을 로드하는 클래스 (GPT 스타일 모델용)
    AutoTokenizer,            # 텍스트를 토큰으로 변환하는 도구
    BitsAndBytesConfig,       # 양자화(Quantization) 설정 - 메모리 절약의 핵심\!
)

# 🔧 PEFT (Parameter-Efficient Fine-Tuning) - LoRA의 핵심
from peft import (
    LoraConfig,               # LoRA 설정 클래스
    get_peft_model,          # 일반 모델을 LoRA 모델로 변환
    prepare_model_for_kbit_training,  # 양자화된 모델을 학습 가능하게 만듦
    TaskType                  # 작업 유형 정의 (예: 언어 생성)
)

# 📚 RAG (Retrieval-Augmented Generation) 관련
from sentence_transformers import SentenceTransformer  # 문장을 벡터로 변환 (임베딩)
import faiss  # Facebook의 벡터 검색 라이브러리 (매우 빠름\!)
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 문서를 청크로 나누기
import PyPDF2  # PDF 파일 읽기

# ⚠️ 경고 메시지 숨기기 (개발 시에는 주석 처리하는 것이 좋음)
warnings.filterwarnings('ignore')

# 📂 프로젝트 디렉토리 구조 설정
# 💡 Path 객체를 사용하면 OS에 관계없이 경로를 다룰 수 있음
BASE_DIR = Path(".")
DATA_DIR = BASE_DIR / "data"
EXTERNAL_DIR = DATA_DIR / "external"  # 외부 문서 (PDF, Excel 등)
AUGMENTED_DIR = DATA_DIR / "augmented"  # 생성된 데이터
VECTORDB_DIR = DATA_DIR / "vectordb"  # RAG 인덱스 저장
MODELS_DIR = BASE_DIR / "models"  # 학습된 모델 저장
RESULTS_DIR = BASE_DIR / "results"  # 실행 결과

# 디렉토리 생성 (exist_ok=True: 이미 있어도 에러 안 남)
for dir_path in [DATA_DIR, EXTERNAL_DIR, AUGMENTED_DIR, VECTORDB_DIR, MODELS_DIR, RESULTS_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

# 🔧 로깅 설정
import logging
logging.basicConfig(
    level=logging.INFO,  # INFO 레벨 이상만 출력
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('training.log'),  # 파일에 저장
        logging.StreamHandler()  # 콘솔에도 출력
    ]
)
logger = logging.getLogger(__name__)

# 🎯 GPU 설정 및 메모리 관리
if torch.cuda.is_available():
    # GPU 사용 가능
    device = torch.device("cuda")
    print(f"🎮 GPU 사용: {torch.cuda.get_device_name(0)}")
    
    # GPU 메모리 정보 출력 (RTX 4090은 24GB)
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    allocated = torch.cuda.memory_allocated() / 1024**3
    print(f"💾 GPU 메모리: {allocated:.2f}GB / {total_memory:.2f}GB")
    
    # 💡 메모리 최적화 팁: 캐시 비우기
    torch.cuda.empty_cache()
else:
    device = torch.device("cpu")
    print("⚠️ GPU를 사용할 수 없습니다. CPU로 실행됩니다.")

# 🌱 재현성을 위한 시드 고정
# 💡 왜 중요한가? 동일한 결과를 재현하기 위해 필수\!
def set_seed(seed: int = 42):
    """
    모든 랜덤 시드를 고정합니다.
    딥러닝에서는 여러 라이브러리가 랜덤을 사용하므로 모두 고정해야 함.
    """
    import random
    
    random.seed(seed)  # Python 기본 random
    np.random.seed(seed)  # NumPy random
    torch.manual_seed(seed)  # PyTorch CPU
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)  # PyTorch GPU
        torch.cuda.manual_seed_all(seed)  # 멀티 GPU
        # Deterministic 연산 강제 (약간 느려질 수 있음)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

print("✅ 환경 설정 완료\!")
print(f"📂 작업 디렉토리: {BASE_DIR.absolute()}")

In [ ]:
# 필수 라이브러리
import os
import json
import torch
import time
import pickle
import logging
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Any
from collections import defaultdict
from tqdm.auto import tqdm

# HuggingFace
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

# PDF 처리
import PyPDF2
from PyPDF2 import PdfReader

# 벡터 DB
import faiss
from sentence_transformers import SentenceTransformer

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# 경로 설정
PROJECT_ROOT = Path("/Users/gunwoo/Downloads/project/ai-dacon")
DATA_DIR = PROJECT_ROOT / "data"
EXTERNAL_DIR = DATA_DIR / "external"
AUGMENTED_DIR = DATA_DIR / "augmented"
VECTORDB_DIR = DATA_DIR / "vectordb"

# 디렉토리 생성
for dir_path in [EXTERNAL_DIR, AUGMENTED_DIR, VECTORDB_DIR]:
    dir_path.mkdir(exist_ok=True, parents=True)

print("✅ 환경 설정 완료!")
print(f"📁 프로젝트 루트: {PROJECT_ROOT}")
print(f"📁 외부 데이터: {EXTERNAL_DIR}")
print(f"📁 증강 데이터: {AUGMENTED_DIR}")

## 1. 🔧 실험 설정 (모든 파라미터 조정 가능)

In [ ]:
# ========================================
# 🎓 실험 설정 클래스 - AI 개발의 핵심: 하이퍼파라미터 관리
# ========================================
#
# 💡 왜 설정 클래스가 중요한가?
#   1. 실험 재현성: 동일한 설정으로 동일한 결과를 얻을 수 있음
#   2. A/B 테스트: 설정만 바꿔가며 성능 비교 가능
#   3. 협업: 팀원들과 설정 공유가 쉬움

class ExperimentConfig:
    """
    실험 설정 클래스 - 모든 파라미터를 한 곳에서 관리
    
    💡 클래스 변수 vs 인스턴스 변수:
    - 여기서는 클래스 변수를 사용 (모든 인스턴스가 공유)
    - 장점: 메모리 효율적, 전역 설정으로 사용하기 좋음
    """
    
    # ===== 모델 선택 =====
    # 🤔 어떤 모델을 선택해야 할까?
    # - 한국어 성능: korean_optimized, exaone이 좋음
    # - 범용성: qwen, mistral이 좋음
    # - 성능: solar가 가장 크고 성능이 좋지만 메모리도 많이 필요
    MODEL_OPTIONS = {
        "korean_optimized": "beomi/llama-2-ko-7b",      # 7B = 70억 개 파라미터
        "solar": "upstage/SOLAR-10.7B-v1.0",            # 10.7B = 107억 개
        "exaone": "LG-AI-EXAONE/EXAONE-3.0-7.8B-Instruct",
        "qwen": "Qwen/Qwen2.5-7B-Instruct",
        "mistral": "mistralai/Mistral-7B-Instruct-v0.2"
    }
    
    MODEL_NAME = MODEL_OPTIONS["korean_optimized"]
    
    # ===== 생성 모드 설정 =====
    # 🎯 각 모드의 특징과 사용 시점:
    GENERATION_MODE = "integrated"  # 기본값: 통합형
    # - "integrated": 질문과 답변을 한 번에 생성 (빠르지만 답변 품질이 낮을 수 있음)
    # - "separated": 질문 먼저, 답변은 RAG 검색 후 생성 (느리지만 정확)
    # - "cot": Chain-of-Thought, 단계별 검증 (매우 느리지만 최고 품질)
    
    # ===== CoT (Chain-of-Thought) 설정 =====
    # 🧠 CoT란? LLM이 "생각의 과정"을 거쳐 답변하도록 하는 기법
    COT_CONFIG = {
        "use_cot": False,           # CoT 사용 여부
        "max_iterations": 3,        # 개선 반복 횟수 (많을수록 품질↑ 속도↓)
        "quality_threshold": 80,    # 품질 기준 (100점 만점)
        
        # 🔍 CoT의 4단계 프로세스:
        # 1. 초기 생성 (Initial Generation)
        # 2. 자가 검증 (Self-Verification) 
        # 3. 개선 (Improvement)
        # 4. 최종 확인 (Final Check)
        "use_self_verification": True,
        "use_improvement": True,
        "use_final_check": True,
        
        # 💾 캐싱: 동일한 입력에 대해 결과 재사용
        "cache_results": True,
        
        # 실험용 세부 설정들
        "verification_strictness": "medium",  # 검증 엄격도
        "focus_areas": ["accuracy", "clarity", "completeness"],  # 집중 영역
        "reasoning_depth": 2,       # 추론 깊이 (1~3)
        "multi_perspective": True,  # 다각도 검증
        "self_critique_level": 2,  # 자기 비판 수준 (0~3)
    }
    
    # CoT 프리셋 - 빠르게 설정 전환 가능
    # 💡 프리셋을 사용하면 일일이 설정을 바꾸지 않아도 됨\!
    COT_PRESETS = {
        "fast": {  # 빠른 프로토타이핑용
            "max_iterations": 1,
            "quality_threshold": 70,
            "use_improvement": False,
        },
        "balanced": {  # 균형잡힌 설정 (추천\!)
            "max_iterations": 2,
            "quality_threshold": 75,
            "use_improvement": True,
        },
        "quality": {  # 품질 우선
            "max_iterations": 4,
            "quality_threshold": 85,
            "use_improvement": True,
            "multi_perspective": True,
        },
        "research": {  # 연구/논문용 (매우 느림)
            "max_iterations": 5,
            "quality_threshold": 90,
            "use_improvement": True,
            "example_generation": True,
        }
    }
    
    CURRENT_COT_PRESET = "balanced"
    
    # 프리셋 적용 로직
    # 💡 Python의 딕셔너리 업데이트 패턴
    if CURRENT_COT_PRESET in COT_PRESETS:
        for key, value in COT_PRESETS[CURRENT_COT_PRESET].items():
            if key in COT_CONFIG:
                COT_CONFIG[key] = value
    
    # ===== 양자화(Quantization) 설정 =====
    # 🎯 양자화란? 모델의 가중치를 압축하여 메모리 사용량을 줄이는 기법
    # - FP32 (32비트) → FP16 (16비트) → INT8 (8비트) → INT4 (4비트)
    # - 4비트 양자화 시 메모리 사용량이 1/8로 줄어듦\!
    USE_QUANTIZATION = True  # RTX 4090 24GB에서는 필수\!
    
    QUANTIZATION_CONFIG = {
        "load_in_4bit": True,  # 4비트로 로드
        "bnb_4bit_quant_type": "nf4",  # NormalFloat4 - 정규분포 기반 양자화
        "bnb_4bit_compute_dtype": torch.float16,  # 계산은 FP16으로
        "bnb_4bit_use_double_quant": True  # 이중 양자화 (더 압축\!)
    }
    
    # ===== 생성 파라미터 (매우 중요\!) =====
    # 🎨 텍스트 생성의 품질을 결정하는 핵심 파라미터들
    GENERATION_PARAMS = {
        "max_new_tokens": 400,      # 생성할 최대 토큰 수 (1토큰 ≈ 0.75단어)
        
        # 🌡️ Temperature: 창의성 조절 (0.1~2.0)
        # - 낮을수록 (0.1): 안전하고 예측 가능한 답변
        # - 높을수록 (1.5): 창의적이지만 때로는 이상한 답변
        "temperature": 0.8,
        
        # 🎯 Sampling 전략들:
        "top_p": 0.9,              # Nucleus sampling - 상위 90% 확률의 토큰만 고려
        "top_k": 50,               # 상위 50개 토큰만 고려
        "do_sample": True,         # 샘플링 사용 (False면 항상 가장 확률 높은 토큰 선택)
        
        # 🔁 반복 방지
        "repetition_penalty": 1.2,  # 이미 나온 토큰의 확률을 낮춤 (1.0 = 패널티 없음)
        
        # 🔍 Beam Search (비활성화됨)
        # - num_beams > 1이면 여러 경로를 동시에 탐색
        # - 품질은 좋아지지만 속도가 느려짐
        "num_beams": 1,
    }
    
    # CoT 모드에서는 단계별로 다른 Temperature 사용
    # 💡 왜? 검증 단계에서는 정확성이 중요하므로 낮은 온도 사용
    COT_GENERATION_PARAMS = {
        "temperature_initial": 0.7,     # 초기 생성 (약간 창의적)
        "temperature_verification": 0.3, # 검증 (매우 보수적)
        "temperature_improvement": 0.5,  # 개선 (중간)
        "temperature_final": 0.3,        # 최종 확인 (보수적)
    }
    
    # ===== RAG (Retrieval-Augmented Generation) 설정 =====
    # 📚 RAG란? 외부 문서를 검색해서 LLM의 답변 품질을 높이는 기법
    RAG_CONFIG = {
        "use_rag": True,           # RAG 사용 여부
        "top_k_retrieval": 3,      # 검색할 문서 수 (많을수록 정보는 많지만 노이즈도 증가)
        
        # 청킹(Chunking) 설정 - 문서를 작은 조각으로 나누기
        # 💡 왜 나누나? 전체 문서는 너무 커서 한 번에 처리 불가
        "chunk_size": 500,         # 각 청크의 크기 (문자 수)
        "chunk_overlap": 50,       # 청크 간 겹침 (문맥 유지용)
        
        # 임베딩 모델 - 텍스트를 벡터로 변환
        # 💡 작은 모델이지만 성능이 좋음 (384차원 벡터)
        "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
        
        "use_cache": True,         # 인덱스 캐싱 (재실행 시 빠름\!)
    }
    
    # ===== 품질 관리 =====
    QUALITY_CONFIG = {
        "min_answer_length": 10,   # 최소 답변 길이 (너무 짧은 답변 방지)
        "max_retry_attempts": 3,   # 실패 시 재시도 횟수
        "quality_threshold": 70,   # 품질 점수 기준 (100점 만점)
        "use_validation": True,    # 품질 검증 사용
        "use_fallback": True,      # 실패 시 대체 방법 사용
    }
    
    # ===== 문제 유형 분포 =====
    # 💡 다양한 유형의 문제를 생성하여 모델의 범용성 향상
    QUESTION_TYPE_DISTRIBUTION = {
        "객관식": 0.30,    # 30% - 선택지에서 고르기
        "주관식": 0.30,    # 30% - 자유롭게 서술
        "단답형": 0.15,    # 15% - 짧은 답변
        "서술형": 0.15,    # 15% - 긴 설명
        "계산형": 0.05,    # 5% - 수치 계산
        "사례분석": 0.05,  # 5% - 실제 사례 분석
    }
    
    # ===== 배치 처리 설정 =====
    # 🚀 대량 데이터 생성 시 중요\!
    BATCH_CONFIG = {
        "batch_size": 4,          # 동시 처리 개수 (메모리와 트레이드오프)
        "target_count": 100,      # 목표 생성 개수
        "max_attempts_ratio": 3,  # 최대 시도 = target * ratio
        "save_interval": 20,      # N개마다 중간 저장 (안전장치\!)
    }
    
    # ===== 실험 모드 =====
    EXPERIMENT_MODE = {
        "verbose": True,          # 상세 로그 출력
        "debug": False,           # 디버그 모드 (더 많은 정보 출력)
        "dry_run": False,         # 실제 실행 없이 테스트만
        "compare_modes": False,   # 여러 모드 비교 실행
        "save_stats": True,       # 통계 저장
    }

# 설정 인스턴스 생성
config = ExperimentConfig()

# 설정 요약 출력
print("🔬 실험 설정 완료\!")
print(f"  📌 모델: {config.MODEL_NAME}")
print(f"  📌 생성 모드: {config.GENERATION_MODE}")
print(f"  📌 CoT 사용: {config.COT_CONFIG['use_cot'] or config.GENERATION_MODE == 'cot'}")
print(f"  📌 CoT 프리셋: {config.CURRENT_COT_PRESET}")
print(f"  📌 양자화: {config.USE_QUANTIZATION}")
print(f"  📌 Temperature: {config.GENERATION_PARAMS['temperature']}")
print(f"  📌 RAG 사용: {config.RAG_CONFIG['use_rag']}")
print(f"  📌 품질 임계값: {config.QUALITY_CONFIG['quality_threshold']}")
print("\n💡 이 설정들을 자유롭게 변경하며 실험해보세요\!")

## 2. RAG 시스템 (문서 검색 및 캐싱)

In [ ]:
# ========================================
# 🎓 RAG 시스템 구현 - 외부 지식을 활용한 AI의 핵심
# ========================================
#
# 💡 RAG(Retrieval-Augmented Generation)란?
#   - LLM의 한계: 학습 데이터에만 의존, 최신 정보 부족
#   - RAG의 해결책: 외부 문서를 검색해서 답변에 활용
#   - 비유: 시험 볼 때 오픈북으로 보는 것과 같음\!

class RAGSystem:
    """
    RAG 시스템 - 문서 검색 및 캐싱 지원
    
    🔍 RAG의 3단계 프로세스:
    1. 문서 준비 (Indexing): PDF/텍스트를 벡터로 변환
    2. 검색 (Retrieval): 질문과 유사한 문서 찾기
    3. 생성 (Generation): 검색된 문서를 참고해 답변 생성
    """
    
    def __init__(self, config: ExperimentConfig):
        self.config = config.RAG_CONFIG
        
        # 🧠 임베딩 모델: 텍스트를 벡터(숫자 배열)로 변환
        # 예: "금융" → [0.1, -0.3, 0.5, ...] (384차원)
        self.embedding_model = None
        
        # 🗂️ 벡터 인덱스: 빠른 유사도 검색을 위한 자료구조
        # FAISS는 Facebook이 만든 초고속 벡터 검색 라이브러리
        self.index = None
        
        # 📚 원본 문서들 저장 (인덱스는 벡터만, 실제 텍스트는 여기에)
        self.documents = []
        
        # 💾 캐시 파일 경로 (한 번 만든 인덱스 재사용)
        self.index_path = VECTORDB_DIR / "index.pkl"
        
    def initialize(self):
        """초기화 - 임베딩 모델 로드 및 인덱스 준비"""
        print("🔍 RAG 시스템 초기화 중...")
        
        # 1. 임베딩 모델 로드
        # 💡 SentenceTransformer: 문장 전체의 의미를 벡터로 표현
        # all-MiniLM-L6-v2는 작지만 성능이 좋은 모델 (50MB)
        self.embedding_model = SentenceTransformer(self.config['embedding_model'])
        
        # 2. 캐시 확인 - 이미 만든 인덱스가 있으면 재사용
        if self.config['use_cache'] and self.index_path.exists():
            # 🚀 캐싱의 효과: 46초 → 0.02초 (2,300배 빨라짐\!)
            self.load_index()
        else:
            # 처음 실행이면 인덱스 구축
            self.build_index()
    
    def build_index(self):
        """인덱스 구축 - PDF 문서를 벡터 DB로 변환"""
        print("📚 문서 인덱스 구축 중...")
        
        # 1. PDF 문서 로드
        documents = self.load_documents()
        
        if not documents:
            print("⚠️ 문서가 없습니다. data/external/ 폴더에 PDF를 추가하세요.")
            return
        
        # 2. 문서를 청크로 분할
        # 💡 왜 분할? LLM의 컨텍스트 길이 제한 때문
        # 청크: 문서를 작은 조각으로 나눈 것
        chunks = self.split_documents(documents)
        print(f"  📄 {len(chunks)}개 청크 생성")
        
        # 3. 각 청크를 벡터로 변환 (임베딩)
        # 🎯 이 과정이 시간이 오래 걸림 (GPU 있으면 빠름)
        print("  🧮 임베딩 생성 중... (첫 실행 시 1-2분 소요)")
        embeddings = self.embedding_model.encode(
            chunks,
            show_progress_bar=True,  # 진행 상황 표시
            batch_size=32  # 배치 처리로 속도 향상
        )
        
        # 4. FAISS 인덱스 생성
        # 💡 FAISS: 수백만 개 벡터도 밀리초 단위로 검색 가능\!
        dimension = embeddings.shape[1]  # 벡터 차원 (보통 384)
        
        # IndexFlatL2: L2 거리(유클리드 거리) 기반 검색
        # 가장 정확하지만 대용량에서는 느릴 수 있음
        self.index = faiss.IndexFlatL2(dimension)
        
        # 벡터 추가
        self.index.add(embeddings.astype('float32'))
        
        # 원본 텍스트 저장 (인덱스는 벡터만 저장하므로)
        self.documents = chunks
        
        # 5. 캐시 저장
        if self.config['use_cache']:
            self.save_index()
            
        print(f"✅ 인덱스 구축 완료\! ({len(chunks)}개 문서)")
        
    def load_documents(self) -> List[str]:
        """PDF 문서 로드"""
        documents = []
        pdf_files = list(EXTERNAL_DIR.glob("*.pdf"))
        
        if not pdf_files:
            # PDF가 없으면 샘플 텍스트라도 사용
            return self._get_sample_documents()
        
        for pdf_path in pdf_files:
            try:
                # PyPDF2로 PDF 읽기
                with open(pdf_path, 'rb') as file:
                    pdf_reader = PyPDF2.PdfReader(file)
                    text = ""
                    
                    # 모든 페이지의 텍스트 추출
                    for page in pdf_reader.pages:
                        text += page.extract_text() + "\n"
                    
                    documents.append(text)
                    print(f"  ✅ {pdf_path.name} 로드 완료")
                    
            except Exception as e:
                print(f"  ❌ {pdf_path.name} 로드 실패: {e}")
                
        return documents
    
    def split_documents(self, documents: List[str]) -> List[str]:
        """
        문서를 청크로 분할
        
        💡 청킹 전략이 RAG 성능에 큰 영향\!
        - 너무 작으면: 문맥 정보 부족
        - 너무 크면: 노이즈 증가, 정확도 하락
        """
        # RecursiveCharacterTextSplitter: 문장 → 단락 → 페이지 순으로 분할
        # 가장 자연스러운 분할 방법
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.config['chunk_size'],      # 각 청크 크기
            chunk_overlap=self.config['chunk_overlap'], # 청크 간 겹침
            length_function=len,  # 길이 계산 함수
            separators=["\n\n", "\n", ".", " ", ""]  # 분할 우선순위
        )
        
        chunks = []
        for doc in documents:
            # 문서를 청크로 분할
            doc_chunks = text_splitter.split_text(doc)
            chunks.extend(doc_chunks)
            
        return chunks
    
    def search(self, query: str, top_k: int = None) -> List[Dict]:
        """
        유사한 문서 검색
        
        🔍 벡터 유사도 검색 과정:
        1. 질문을 벡터로 변환
        2. 모든 문서 벡터와 거리 계산
        3. 가장 가까운 k개 선택
        """
        if self.index is None or not self.documents:
            print("⚠️ 인덱스가 없습니다. 먼저 initialize()를 실행하세요.")
            return []
        
        top_k = top_k or self.config['top_k_retrieval']
        
        # 1. 질문을 벡터로 변환
        # 💡 질문과 문서를 같은 벡터 공간에 매핑
        query_embedding = self.embedding_model.encode([query])
        
        # 2. 가장 유사한 문서 검색
        # D: 거리(작을수록 유사), I: 인덱스
        distances, indices = self.index.search(
            query_embedding.astype('float32'), 
            top_k
        )
        
        # 3. 결과 정리
        results = []
        for idx, distance in zip(indices[0], distances[0]):
            if idx < len(self.documents):  # 범위 체크
                results.append({
                    'text': self.documents[idx],
                    'distance': float(distance),  # L2 거리
                    'similarity': 1 / (1 + float(distance))  # 유사도 점수로 변환
                })
        
        # 유사도 순으로 정렬
        results.sort(key=lambda x: x['similarity'], reverse=True)
        
        return results
    
    def save_index(self):
        """인덱스 캐시 저장"""
        print("💾 인덱스 캐시 저장 중...")
        
        # pickle로 저장 (Python 객체 직렬화)
        cache_data = {
            'index': faiss.serialize_index(self.index),  # FAISS 인덱스
            'documents': self.documents,  # 원본 텍스트
            'timestamp': datetime.now().isoformat()  # 생성 시간
        }
        
        with open(self.index_path, 'wb') as f:
            pickle.dump(cache_data, f)
            
        print(f"✅ 캐시 저장 완료: {self.index_path}")
        
    def load_index(self):
        """인덱스 캐시 로드"""
        print("📂 캐시된 인덱스 로드 중...")
        
        with open(self.index_path, 'rb') as f:
            cache_data = pickle.load(f)
            
        # FAISS 인덱스 복원
        self.index = faiss.deserialize_index(cache_data['index'])
        self.documents = cache_data['documents']
        
        print(f"✅ 캐시 로드 완료\! ({len(self.documents)}개 문서)")
        print(f"  생성 시간: {cache_data['timestamp']}")
        
    def _get_sample_documents(self) -> List[str]:
        """PDF가 없을 때 사용할 샘플 문서"""
        # 💡 실제 프로젝트에서는 반드시 실제 문서를 사용하세요\!
        return [
            """바젤III 규제는 2008년 금융위기 이후 도입된 국제 은행 규제 프레임워크입니다.
            주요 내용으로는 자본 적정성 강화, 레버리지 비율 도입, 유동성 규제 신설 등이 있습니다.
            보통주자본비율은 4.5%, Tier1 자본비율은 6%, 총자본비율은 8% 이상을 유지해야 합니다.""",
            
            """금융보안원(FSI)은 국내 금융 IT 보안을 총괄하는 전문기관입니다.
            주요 업무로는 금융권 사이버 보안 강화, 전자금융거래 안전성 확보,
            금융회사 보안 수준 평가 및 점검 등이 있습니다.""",
            
            """파생상품은 기초자산의 가격 변동에 따라 가치가 결정되는 금융상품입니다.
            선물(Futures), 옵션(Options), 스왑(Swaps) 등이 대표적이며,
            위험 헤지(Hedging)와 투기(Speculation) 목적으로 활용됩니다."""
        ]

print("✅ RAG 시스템 코드 정의 완료\!")
print("💡 사용법:")
print("  rag = RAGSystem(config)")
print("  rag.initialize()  # 인덱스 구축 또는 로드")
print("  results = rag.search('바젤III 자본비율')  # 검색")

## 3. 향상된 프롬프트 템플릿 (통합형/분리형)

In [ ]:
class EnhancedPromptTemplates:
    """
    답변 생성이 보장된 향상된 프롬프트 템플릿
    통합형과 분리형 모두 지원
    """
    
    def __init__(self, style: str = "detailed"):
        self.style = style  # simple, detailed, expert
        self.templates = self._init_templates()
    
    def _init_templates(self) -> Dict:
        """스타일별 템플릿 초기화"""
        
        if self.style == "simple":
            # 간단한 프롬프트 (빠르지만 품질 낮음)
            return self._get_simple_templates()
        elif self.style == "expert":
            # 전문가 프롬프트 (느리지만 품질 높음)
            return self._get_expert_templates()
        else:
            # 기본: 상세 프롬프트 (균형)
            return self._get_detailed_templates()
    
    def _get_detailed_templates(self) -> Dict:
        """상세 템플릿 (기본)"""
        return {
            # ===== 통합형 템플릿 (질문+답변 동시) =====
            "integrated": {
                "객관식": """당신은 금융보안원 FSKU 출제위원입니다.

참고 문서:
{context}

위 내용을 바탕으로 FSKU 객관식 문제를 생성하세요.

요구사항:
1. 문제는 명확하고 구체적이어야 함
2. 4개의 선택지를 제시
3. 정답은 반드시 하나만 존재
4. 오답도 그럴듯해야 함

형식 (반드시 이 형식을 지켜주세요):
문제: [구체적인 질문]
① [선택지1]
② [선택지2]
③ [선택지3]
④ [선택지4]
정답: [정답 번호와 내용]
해설: [정답 선택 이유]""",

                "주관식": """당신은 금융보안원 FSKU 출제위원입니다.

참고 문서:
{context}

위 내용을 바탕으로 FSKU 주관식 문제를 생성하세요.

요구사항:
1. 2-3문장으로 답할 수 있는 문제
2. 구체적이고 명확한 답변이 가능해야 함
3. 핵심 개념을 묻는 문제여야 함

형식 (반드시 이 형식을 지켜주세요):
문제: [구체적인 질문]
정답: [2-3문장의 완전한 답변]
핵심 키워드: [중요 키워드 3-5개]
채점 기준: [평가 기준]""",

                "단답형": """당신은 금융보안원 FSKU 출제위원입니다.

참고 문서:
{context}

위 내용을 바탕으로 FSKU 단답형 문제를 생성하세요.

요구사항:
1. 한 단어나 짧은 구로 답할 수 있는 문제
2. 명확한 정답이 존재해야 함
3. 용어, 수치, 개념명을 묻는 문제

형식 (반드시 이 형식을 지켜주세요):
문제: [구체적인 질문]
정답: [단답형 정답]
허용 답안: [유사 정답들]""",

                "서술형": """당신은 금융보안원 FSKU 출제위원입니다.

참고 문서:
{context}

위 내용을 바탕으로 FSKU 서술형 문제를 생성하세요.

요구사항:
1. 상세한 설명이 필요한 문제
2. 논리적 전개가 중요한 답변
3. 5문장 이상의 답변 필요

형식 (반드시 이 형식을 지켜주세요):
문제: [구체적인 질문]
모범 답안: [5문장 이상의 상세한 답변]
핵심 평가 요소: [평가할 요소들]""",
            },
            
            # ===== 분리형 템플릿 - 질문 생성용 =====
            "question": {
                "객관식": """금융보안원 FSKU 시험을 위한 객관식 문제를 생성하세요.

참고 문서:
{context}

위 내용에서 중요한 개념을 선택하여 4지선다 문제를 만드세요.
문제와 선택지만 생성하고, 정답은 생성하지 마세요.

형식:
문제: [질문]
① [선택지1]
② [선택지2]
③ [선택지3]
④ [선택지4]""",

                "주관식": """금융보안원 FSKU 시험을 위한 주관식 문제를 생성하세요.

참고 문서:
{context}

위 내용에서 중요한 개념을 설명하도록 요구하는 문제를 만드세요.
문제만 생성하고, 답변은 생성하지 마세요.

형식:
문제: [서술형 질문]""",
            },
            
            # ===== 분리형 템플릿 - 답변 생성용 =====
            "answer": {
                "객관식": """다음 문제의 정답을 선택하고 설명하세요.

참고 문서:
{context}

문제:
{question}

위 문제의 정답을 선택하고, 그 이유를 설명하세요.

형식:
정답: [정답 번호와 내용]
해설: [선택 이유와 다른 선택지가 틀린 이유]""",

                "주관식": """다음 문제에 대한 완전하고 정확한 답변을 작성하세요.

참고 문서:
{context}

문제:
{question}

위 문제에 대해 2-3문장으로 명확하게 답변하세요.

형식:
정답: [완전한 답변]
핵심 키워드: [중요 용어들]""",
            }
        }
    
    def _get_simple_templates(self) -> Dict:
        """간단한 템플릿 (빠름)"""
        return {
            "integrated": {
                "객관식": """참고: {context}

객관식 문제 생성:
문제:
①
②
③
④
정답:""",
                "주관식": """참고: {context}

주관식 문제 생성:
문제:
정답:""",
            },
            "question": {
                "객관식": """참고: {context}
객관식 문제만:""",
                "주관식": """참고: {context}
주관식 문제만:""",
            },
            "answer": {
                "객관식": """문제: {question}
정답:""",
                "주관식": """문제: {question}
정답:""",
            }
        }
    
    def _get_expert_templates(self) -> Dict:
        """전문가 템플릿 (고품질)"""
        return {
            "integrated": {
                "객관식": """[전문가 모드]
당신은 20년 경력의 FSKU 출제위원장입니다.

참고 문서:
{context}

다음 기준에 따라 최고 품질의 객관식 문제를 생성하세요:
1. Bloom's Taxonomy 상위 수준 (분석, 평가, 창조)
2. 실무 상황 기반
3. 함정 선택지 포함
4. 명확한 변별력

필수 포함 요소:
- 문제 상황 설정
- 구체적 조건 제시
- 4개 선택지 (모두 그럴듯함)
- 정답과 상세 해설

형식:
문제: [상황 설정 + 구체적 질문]
① [선택지1 - 매력적인 오답]
② [선택지2 - 부분적으로 맞는 오답]
③ [선택지3 - 정답 또는 오답]
④ [선택지4 - 혼동하기 쉬운 오답]
정답: [번호와 내용]
해설: [각 선택지별 상세 설명]
난이도: [상/중/하]
출제 의도: [평가하고자 하는 역량]""",
                
                # 다른 유형들도 유사하게 전문가 수준으로...
            },
            # question, answer도 전문가 수준으로 확장...
        }
    
    def get_template(self, mode: str, question_type: str, template_type: str = "integrated") -> str:
        """
        템플릿 가져오기
        
        Args:
            mode: generation mode (integrated/separated)
            question_type: 문제 유형
            template_type: integrated/question/answer
        """
        if mode == "integrated":
            return self.templates["integrated"].get(
                question_type, 
                self.templates["integrated"]["주관식"]
            )
        else:  # separated
            return self.templates[template_type].get(
                question_type,
                self.templates[template_type]["주관식"]
            )

print("✅ 프롬프트 템플릿 클래스 정의 완료")
print("📝 스타일 옵션: simple, detailed, expert")
print("📝 모드: integrated (통합형), separated (분리형)")

In [ ]:
# ========================================
# 🎓 데이터 생성기 - LLM을 활용한 학습 데이터 자동 생성
# ========================================
#
# 💡 왜 데이터 생성이 필요한가?
#   - 문제: 고품질 학습 데이터 부족 (특히 한국어 금융 분야)
#   - 해결: LLM을 사용해 자동으로 Q&A 쌍 생성
#   - 주의: 생성된 데이터의 품질 검증 필수\!

class AnswerGuaranteedGenerator:
    """
    답변 생성이 보장된 데이터 생성기
    
    🎯 핵심 기능:
    1. 통합형/분리형/CoT 모드 지원
    2. 답변 없을 시 3단계 폴백 시스템
    3. 품질 검증 포함
    """
    
    def __init__(self, config: ExperimentConfig, rag_system: RAGSystem = None):
        self.config = config
        self.generation_mode = config.GENERATION_MODE
        self.rag = rag_system
        
        # 🤖 모델과 토크나이저 (나중에 로드)
        self.model = None
        self.tokenizer = None
        self.model_loaded = False
        
        # 📊 통계 추적 (성능 분석용)
        self.stats = {
            'total_attempts': 0,      # 총 시도 횟수
            'successful': 0,           # 성공 횟수
            'failed': 0,              # 실패 횟수
            'retry_count': 0,         # 재시도 횟수
            'with_answer': 0,         # 답변 있는 경우
            'without_answer': 0,      # 답변 없는 경우
            'fallback_used': 0,       # 폴백 사용 횟수
        }
    
    def load_model(self):
        """
        모델 로드 - 메모리 효율적으로 LLM 로드
        
        💡 모델 로드 시 고려사항:
        1. 메모리 제한: RTX 4090은 24GB
        2. 양자화 사용: 4bit로 압축하면 7B 모델도 로드 가능
        3. device_map="auto": 자동으로 GPU/CPU 분배
        """
        if self.model_loaded:
            return  # 이미 로드됨
        
        print(f"🚀 모델 로딩: {self.config.MODEL_NAME}")
        print(f"  양자화: {self.config.USE_QUANTIZATION}")
        
        try:
            # 1. 토크나이저 로드
            # 💡 토크나이저: 텍스트 ↔ 토큰 변환
            # 예: "안녕하세요" → [1234, 5678, 9012]
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.config.MODEL_NAME,
                trust_remote_code=True  # 커스텀 코드 허용 (일부 모델 필요)
            )
            
            # 패딩 토큰 설정 (없으면 EOS 토큰 사용)
            # 💡 패딩: 배치 처리 시 길이를 맞추기 위한 특수 토큰
            if not self.tokenizer.pad_token:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            
            # 2. 모델 로드 (양자화 여부에 따라 다르게)
            if self.config.USE_QUANTIZATION:
                # 🔥 QLoRA 방식: 4bit 양자화
                # 메모리 사용량: 7B 모델 → 약 4GB
                bnb_config = BitsAndBytesConfig(**self.config.QUANTIZATION_CONFIG)
                
                self.model = AutoModelForCausalLM.from_pretrained(
                    self.config.MODEL_NAME,
                    quantization_config=bnb_config,  # 양자화 설정
                    device_map="auto",  # GPU/CPU 자동 분배
                    trust_remote_code=True
                )
            else:
                # 일반 방식: FP16 (반정밀도)
                # 메모리 사용량: 7B 모델 → 약 14GB
                self.model = AutoModelForCausalLM.from_pretrained(
                    self.config.MODEL_NAME,
                    torch_dtype=torch.float16,  # FP32 → FP16 (메모리 절반)
                    device_map="auto",
                    trust_remote_code=True
                )
            
            self.model_loaded = True
            print("✅ 모델 로드 완료\!")
            
            # GPU 메모리 사용량 출력
            if torch.cuda.is_available():
                memory = torch.cuda.memory_allocated() / 1024**3
                print(f"💾 GPU 메모리 사용: {memory:.2f}GB")
                
        except Exception as e:
            logger.error(f"모델 로드 실패: {e}")
            # 💡 일반적인 오류 원인:
            # 1. 메모리 부족 → 양자화 사용 또는 더 작은 모델
            # 2. 모델명 오타 → MODEL_NAME 확인
            # 3. 인터넷 연결 → 첫 다운로드 시 필요
            raise
    
    def generate_text(self, prompt: str, max_tokens: int = None) -> str:
        """
        텍스트 생성 - LLM의 핵심 기능
        
        🎨 생성 과정:
        1. 프롬프트 → 토큰화
        2. 모델 추론 (Forward pass)
        3. 토큰 샘플링 (Temperature, Top-p 등 적용)
        4. 토큰 → 텍스트 변환
        """
        if not self.model_loaded:
            self.load_model()
        
        max_tokens = max_tokens or self.config.GENERATION_PARAMS['max_new_tokens']
        
        # 1. 토큰화
        # 💡 return_tensors="pt": PyTorch 텐서로 반환
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,  # 너무 길면 자르기
            max_length=2000   # 최대 입력 길이
        )
        
        # GPU로 이동 (가능한 경우)
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}
        
        # 2. 생성 (추론)
        # 💡 torch.no_grad(): 그래디언트 계산 비활성화 (메모리 절약)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,  # input_ids, attention_mask 등
                max_new_tokens=max_tokens,
                
                # 샘플링 파라미터들 (품질 결정\!)
                temperature=self.config.GENERATION_PARAMS['temperature'],
                top_p=self.config.GENERATION_PARAMS['top_p'],
                top_k=self.config.GENERATION_PARAMS['top_k'],
                do_sample=self.config.GENERATION_PARAMS['do_sample'],
                repetition_penalty=self.config.GENERATION_PARAMS['repetition_penalty'],
                
                # 특수 토큰 ID
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
            )
        
        # 3. 디코딩 (토큰 → 텍스트)
        # 입력 부분 제외하고 생성된 부분만 추출
        generated = self.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:],  # 입력 길이 이후부터
            skip_special_tokens=True  # <pad>, <eos> 등 제거
        )
        
        return generated.strip()
    
    def generate_qa_pair(self, context: str, question_type: str = "주관식") -> Optional[Dict]:
        """
        QA 쌍 생성 - 메인 함수
        
        🔄 생성 모드별 차이:
        1. integrated: 한 번에 Q&A 생성 (빠름)
        2. separated: Q 생성 → RAG 검색 → A 생성 (정확)
        3. cot: 4단계 검증 과정 (최고 품질)
        """
        self.stats['total_attempts'] += 1
        
        try:
            if self.generation_mode == "integrated":
                # 통합형: 프롬프트 하나로 Q&A 동시 생성
                return self._generate_integrated(context, question_type)
                
            elif self.generation_mode == "separated":
                # 분리형: 질문 먼저, 답변은 따로
                return self._generate_separated(context, question_type)
                
            elif self.generation_mode == "cot":
                # CoT: Chain-of-Thought로 단계별 검증
                # 별도 클래스에서 처리 (ChainOfThoughtGenerator)
                pass
                
        except Exception as e:
            logger.error(f"QA 생성 오류: {e}")
            self.stats['failed'] += 1
            return None
    
    def _generate_integrated(self, context: str, question_type: str) -> Optional[Dict]:
        """
        통합형 생성 - 한 번에 Q&A 생성
        
        장점: 빠름, 일관성 있음
        단점: 답변 품질이 낮을 수 있음
        """
        # 프롬프트 구성
        # 💡 프롬프트 엔지니어링이 품질의 80%를 결정\!
        prompt = f"""당신은 한국 금융감독원의 FSKU 시험 출제위원입니다.
다음 문서를 참고하여 {question_type} 문제와 답변을 생성하세요.

### 참고 문서:
{context[:1000]}  # 너무 길면 잘라서 사용

### 생성 지침:
1. 문제는 명확하고 구체적으로
2. 답변은 완전하고 정확하게
3. 금융 전문 용어를 적절히 사용
4. 실무에서 중요한 내용 위주로

### 형식:
문제: [여기에 질문 작성]
정답: [여기에 답변 작성]

### 생성:"""
        
        # 텍스트 생성
        generated = self.generate_text(prompt)
        
        # 파싱 (생성된 텍스트에서 Q&A 추출)
        result = self._parse_qa(generated)
        
        if result:
            self.stats['successful'] += 1
            if result.get('answer'):
                self.stats['with_answer'] += 1
            else:
                self.stats['without_answer'] += 1
                
        return result
    
    def _generate_separated(self, context: str, question_type: str) -> Optional[Dict]:
        """
        분리형 생성 - 질문과 답변을 따로 생성
        
        🔍 개선된 프로세스:
        1. 컨텍스트로 질문 생성
        2. 생성된 질문으로 RAG 재검색 ← 핵심\!
        3. 원본 + 검색 결과로 답변 생성
        """
        # 1단계: 질문 생성
        question_prompt = f"""문서를 읽고 {question_type} 질문을 하나만 생성하세요.

문서: {context[:500]}

질문:"""
        
        question = self.generate_text(question_prompt, max_tokens=100)
        
        if not question:
            return None
        
        # 2단계: RAG 검색 (생성된 질문 기반)
        # 💡 이것이 분리형의 핵심 개선\!
        enhanced_context = context
        if self.rag:
            # 질문으로 관련 문서 검색
            retrieved = self.rag.search(question, top_k=3)
            if retrieved:
                # 검색 결과를 컨텍스트에 추가
                rag_context = "\n".join([r['text'][:200] for r in retrieved])
                enhanced_context = f"{context}\n\n관련 정보:\n{rag_context}"
        
        # 3단계: 답변 생성
        answer_prompt = f"""다음 질문에 대한 정확한 답변을 작성하세요.

참고 자료:
{enhanced_context[:800]}

질문: {question}

답변:"""
        
        answer = self.generate_text(answer_prompt, max_tokens=300)
        
        return {
            'question': question.strip(),
            'answer': answer.strip(),
            'context': context[:500],
            'question_type': question_type,
            'generation_mode': 'separated',
            'rag_used': self.rag is not None
        }
    
    def _parse_qa(self, text: str) -> Optional[Dict]:
        """
        생성된 텍스트에서 Q&A 추출
        
        💡 파싱은 의외로 까다로움\!
        LLM이 항상 정확한 형식으로 생성하지 않기 때문
        """
        result = {}
        
        # 다양한 형식 처리
        # LLM마다 선호하는 형식이 다름
        question_markers = ['문제:', '질문:', 'Q:', 'Question:']
        answer_markers = ['정답:', '답변:', '답:', 'A:', 'Answer:']
        
        # 질문 추출
        for marker in question_markers:
            if marker in text:
                parts = text.split(marker, 1)[1]
                # 답변 마커까지만 추출
                for ans_marker in answer_markers:
                    if ans_marker in parts:
                        result['question'] = parts.split(ans_marker)[0].strip()
                        break
                break
        
        # 답변 추출
        for marker in answer_markers:
            if marker in text:
                answer_part = text.split(marker, 1)[1]
                # 다음 섹션이나 줄바꿈까지
                result['answer'] = answer_part.split('\n\n')[0].strip()
                break
        
        # 둘 다 있어야 유효
        if 'question' in result and 'answer' in result:
            return result
        
        return None
    
    def get_stats(self) -> Dict:
        """통계 반환 - 성능 분석용"""
        total = self.stats['total_attempts']
        if total == 0:
            return self.stats
        
        # 성공률 계산
        success_rate = self.stats['successful'] / total * 100
        answer_rate = self.stats['with_answer'] / max(self.stats['successful'], 1) * 100
        
        return {
            **self.stats,
            'success_rate': f"{success_rate:.1f}%",
            'answer_rate': f"{answer_rate:.1f}%",
            'fallback_rate': f"{self.stats['fallback_used'] / total * 100:.1f}%"
        }

print("✅ 데이터 생성기 정의 완료\!")
print("💡 각 모드의 사용 시점:")
print("  - integrated: 빠른 프로토타이핑")
print("  - separated: 정확한 답변이 필요할 때")
print("  - cot: 최고 품질이 필요할 때")

In [ ]:
class AnswerGuaranteedGenerator:
    """
    답변 생성이 보장된 데이터 생성기
    - 통합형과 분리형 모두 지원
    - 답변 없을 시 3단계 폴백 시스템
    - 품질 검증 포함
    """
    
    def __init__(self, config: ExperimentConfig, rag_system: RAGSystem = None):
        """
        초기화
        
        Args:
            config: 실험 설정
            rag_system: RAG 시스템 (선택적)
        """
        self.config = config
        self.generation_mode = config.GENERATION_MODE
        self.rag = rag_system
        
        # 프롬프트 템플릿
        self.prompts = EnhancedPromptTemplates(config.PROMPT_STYLE)
        
        # 모델 초기화
        self.model = None
        self.tokenizer = None
        self.model_loaded = False
        
        # 통계
        self.stats = {
            'total_attempts': 0,
            'successful': 0,
            'failed': 0,
            'retry_count': 0,
            'with_answer': 0,
            'without_answer': 0,
            'fallback_used': 0,
            'mode_stats': {'integrated': 0, 'separated': 0}
        }
    
    def load_model(self):
        """모델 로드"""
        if self.model_loaded:
            return
        
        print(f"🚀 모델 로딩: {self.config.MODEL_NAME}")
        print(f"  양자화: {self.config.USE_QUANTIZATION}")
        
        try:
            # 토크나이저 로드
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.config.MODEL_NAME,
                trust_remote_code=True
            )
            
            if not self.tokenizer.pad_token:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            
            # 모델 로드
            if self.config.USE_QUANTIZATION:
                bnb_config = BitsAndBytesConfig(**self.config.QUANTIZATION_CONFIG)
                
                self.model = AutoModelForCausalLM.from_pretrained(
                    self.config.MODEL_NAME,
                    quantization_config=bnb_config,
                    device_map="auto",
                    trust_remote_code=True
                )
            else:
                self.model = AutoModelForCausalLM.from_pretrained(
                    self.config.MODEL_NAME,
                    torch_dtype=torch.float16,
                    device_map="auto",
                    trust_remote_code=True
                )
            
            self.model_loaded = True
            print("✅ 모델 로드 완료!")
            
            # GPU 메모리 사용량
            if torch.cuda.is_available():
                memory = torch.cuda.memory_allocated() / 1024**3
                print(f"💾 GPU 메모리: {memory:.2f}GB")
                
        except Exception as e:
            logger.error(f"모델 로드 실패: {e}")
            raise
    
    def generate_text(self, prompt: str, max_tokens: int = None) -> str:
        """텍스트 생성"""
        if not self.model_loaded:
            self.load_model()
        
        max_tokens = max_tokens or self.config.GENERATION_PARAMS['max_new_tokens']
        
        # 토큰화
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=2000
        )
        
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}
        
        # 생성
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=self.config.GENERATION_PARAMS['temperature'],
                top_p=self.config.GENERATION_PARAMS['top_p'],
                top_k=self.config.GENERATION_PARAMS['top_k'],
                do_sample=self.config.GENERATION_PARAMS['do_sample'],
                repetition_penalty=self.config.GENERATION_PARAMS['repetition_penalty'],
                num_beams=self.config.GENERATION_PARAMS['num_beams'],
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id
            )
        
        # 디코딩
        generated = self.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:],
            skip_special_tokens=True
        )
        
        return generated.strip()
    
    def generate_integrated(self, context: str, question_type: str) -> Optional[Dict]:
        """
        통합형 생성 (질문+답변 동시)
        빠르고 일관성 있음
        """
        if self.config.EXPERIMENT_MODE['verbose']:
            print(f"  [통합형] {question_type} 생성 중...")
        
        # 프롬프트 생성
        prompt_template = self.prompts.get_template(
            "integrated", question_type, "integrated"
        )
        prompt = prompt_template.format(context=context[:1000])
        
        # 재시도 로직
        max_retry = self.config.QUALITY_CONFIG['max_retry_attempts']
        
        for attempt in range(max_retry):
            generated = self.generate_text(prompt)
            result = self._parse_integrated_result(generated, question_type)
            
            if result and result.get('answer'):
                # 답변 길이 확인
                if len(result['answer']) >= self.config.QUALITY_CONFIG['min_answer_length']:
                    return result
            
            self.stats['retry_count'] += 1
            
            if self.config.EXPERIMENT_MODE['verbose']:
                print(f"    ⚠️ 답변 생성 실패, 재시도 {attempt + 1}/{max_retry}")
        
        return None
    
    def generate_separated(self, context: str, question_type: str) -> Optional[Dict]:
        """
        분리형 생성 (질문 먼저, 답변 나중에)
        느리지만 더 정확한 답변 가능
        """
        if self.config.EXPERIMENT_MODE['verbose']:
            print(f"  [분리형] {question_type} 생성 중...")
        
        # 1단계: 질문 생성
        if self.config.EXPERIMENT_MODE['verbose']:
            print("    1/2. 질문 생성...")
        
        q_template = self.prompts.get_template(
            "separated", question_type, "question"
        )
        q_prompt = q_template.format(context=context[:1000])
        
        question_text = self.generate_text(q_prompt, max_tokens=200)
        parsed_question = self._parse_question(question_text, question_type)
        
        if not parsed_question:
            if self.config.EXPERIMENT_MODE['verbose']:
                print("    ❌ 질문 생성 실패")
            return None
        
        # 2단계: 답변 생성
        if self.config.EXPERIMENT_MODE['verbose']:
            print("    2/2. 답변 생성...")
        
        a_template = self.prompts.get_template(
            "separated", question_type, "answer"
        )
        a_prompt = a_template.format(
            context=context[:1000],
            question=parsed_question['question']
        )
        
        answer_text = self.generate_text(a_prompt, max_tokens=300)
        parsed_answer = self._parse_answer(answer_text, question_type)
        
        if not parsed_answer and self.config.QUALITY_CONFIG['use_fallback']:
            # 답변 생성 실패 시 폴백
            if self.config.EXPERIMENT_MODE['verbose']:
                print("    ⚠️ 폴백 답변 생성...")
            
            parsed_answer = self._generate_fallback_answer(
                context, parsed_question['question'], question_type
            )
            self.stats['fallback_used'] += 1
        
        # 결과 병합
        result = {**parsed_question, **parsed_answer}
        return result
    
    def generate_qa_pair(self, context: str, question_type: str = "주관식") -> Optional[Dict]:
        """
        QA 쌍 생성 (메인 메서드)
        
        Args:
            context: 참고 문서 내용
            question_type: 문제 유형
        
        Returns:
            생성된 QA 쌍
        """
        self.stats['total_attempts'] += 1
        
        try:
            # RAG 사용 시 컨텍스트 보강
            if self.rag and self.config.RAG_CONFIG['use_rag']:
                # 질문 유형을 쿼리로 사용
                query = f"{question_type} 문제 생성을 위한 {context[:100]}"
                retrieved = self.rag.search(query)
                
                if retrieved:
                    # 검색된 문서 추가
                    additional_context = "\n".join([r['text'][:200] for r in retrieved[:2]])
                    context = f"{context}\n\n관련 문서:\n{additional_context}"
            
            # 생성 모드에 따라 분기
            if self.generation_mode == "integrated":
                result = self.generate_integrated(context, question_type)
                self.stats['mode_stats']['integrated'] += 1
            else:
                result = self.generate_separated(context, question_type)
                self.stats['mode_stats']['separated'] += 1
            
            if result:
                # 답변 존재 확인
                if result.get('answer'):
                    self.stats['with_answer'] += 1
                else:
                    self.stats['without_answer'] += 1
                    
                    # 답변 없으면 기본 답변 생성
                    if self.config.QUALITY_CONFIG['use_fallback']:
                        result['answer'] = self._generate_basic_answer(
                            context, result.get('question', ''), question_type
                        )
                
                # 품질 검증
                if self.config.QUALITY_CONFIG['use_validation']:
                    quality = self._validate_quality(result)
                    result['quality_score'] = quality
                    
                    # 품질 임계값 확인
                    if quality < self.config.QUALITY_CONFIG['quality_threshold']:
                        if self.config.EXPERIMENT_MODE['verbose']:
                            print(f"    ⚠️ 품질 미달: {quality}/100")
                        # 품질 미달이어도 일단 반환 (나중에 필터링)
                
                # 메타데이터 추가
                result['context'] = context[:500]  # 컨텍스트 일부만 저장
                result['question_type'] = question_type
                result['generation_mode'] = self.generation_mode
                result['model'] = self.config.MODEL_NAME
                result['timestamp'] = datetime.now().isoformat()
                
                self.stats['successful'] += 1
                return result
            else:
                self.stats['failed'] += 1
                return None
                
        except Exception as e:
            logger.error(f"생성 오류: {e}")
            self.stats['failed'] += 1
            return None
    
    def _parse_integrated_result(self, text: str, question_type: str) -> Optional[Dict]:
        """통합 결과 파싱"""
        result = {}
        
        # 문제 추출
        if '문제:' in text:
            parts = text.split('문제:', 1)[1]
            
            # 정답 구분자 찾기
            if '정답:' in parts:
                result['question'] = parts.split('정답:')[0].strip()
            elif '모범 답안:' in parts:
                result['question'] = parts.split('모범 답안:')[0].strip()
            elif '①' in parts:  # 객관식
                result['question'] = parts.split('①')[0].strip()
            else:
                result['question'] = parts.split('\n')[0].strip()
        
        # 정답 추출 (중요!)
        if '정답:' in text:
            answer_part = text.split('정답:', 1)[1]
            
            # 다음 섹션까지 추출
            for delimiter in ['해설:', '핵심', '채점', '\n\n', '\n문제:']:
                if delimiter in answer_part:
                    result['answer'] = answer_part.split(delimiter)[0].strip()
                    break
            else:
                result['answer'] = answer_part.strip()
        
        elif '모범 답안:' in text:
            answer_part = text.split('모범 답안:', 1)[1]
            result['answer'] = answer_part.split('\n\n')[0].strip()
        
        # 추가 정보
        if '해설:' in text:
            result['explanation'] = text.split('해설:', 1)[1].strip()
        
        if '핵심 키워드:' in text:
            result['keywords'] = text.split('핵심 키워드:', 1)[1].split('\n')[0].strip()
        
        # 객관식 선택지
        if question_type == "객관식":
            choices = []
            for marker in ['①', '②', '③', '④']:
                if marker in text:
                    idx = text.index(marker)
                    choice_text = text[idx:]
                    choice_line = choice_text.split('\n')[0]
                    choices.append(choice_line)
            if choices:
                result['choices'] = choices
        
        # 검증: 질문과 답변 모두 있어야 함
        if 'question' in result and 'answer' in result:
            return result
        
        return None
    
    def _parse_question(self, text: str, question_type: str) -> Optional[Dict]:
        """질문 파싱 (분리형용)"""
        result = {}
        
        if '문제:' in text:
            question_part = text.split('문제:', 1)[1]
            
            if question_type == "객관식":
                # 선택지 분리
                if '①' in question_part:
                    result['question'] = question_part.split('①')[0].strip()
                    
                    # 선택지 추출
                    choices = []
                    for marker in ['①', '②', '③', '④']:
                        if marker in text:
                            idx = text.index(marker)
                            choice_text = text[idx:]
                            choice_line = choice_text.split('\n')[0]
                            choices.append(choice_line)
                    result['choices'] = choices
                else:
                    result['question'] = question_part.strip()
            else:
                result['question'] = question_part.strip()
        else:
            # 문제: 마커가 없는 경우
            result['question'] = text.strip()
        
        return result if 'question' in result else None
    
    def _parse_answer(self, text: str, question_type: str) -> Optional[Dict]:
        """답변 파싱 (분리형용)"""
        result = {}
        
        if '정답:' in text:
            answer_part = text.split('정답:', 1)[1]
            
            # 다음 섹션까지 추출
            for delimiter in ['해설:', '핵심 키워드:', '채점 기준:', '\n\n']:
                if delimiter in answer_part:
                    result['answer'] = answer_part.split(delimiter)[0].strip()
                    break
            else:
                result['answer'] = answer_part.strip()
        else:
            # 정답: 마커가 없는 경우
            result['answer'] = text.strip()
        
        # 추가 정보
        if '해설:' in text:
            result['explanation'] = text.split('해설:', 1)[1].strip()
        
        if '핵심 키워드:' in text:
            result['keywords'] = text.split('핵심 키워드:', 1)[1].split('\n')[0].strip()
        
        return result if 'answer' in result else None
    
    def _generate_fallback_answer(self, context: str, question: str, question_type: str) -> Dict:
        """폴백 답변 생성 (답변 생성 실패 시)"""
        if self.config.EXPERIMENT_MODE['verbose']:
            print("      🔄 폴백 답변 생성 중...")
        
        # 간단한 프롬프트로 답변만 생성
        simple_prompt = f"""다음 질문에 답하세요.

참고: {context[:400]}

질문: {question}

답변:"""
        
        answer = self.generate_text(simple_prompt, max_tokens=200)
        
        return {
            'answer': answer if answer else "[답변 생성 실패 - 수동 작성 필요]",
            'fallback': True
        }
    
    def _generate_basic_answer(self, context: str, question: str, question_type: str) -> str:
        """기본 답변 생성 (매우 간단)"""
        # 컨텍스트에서 관련 부분 추출
        sentences = context.split('.')
        
        # 질문과 가장 관련있는 문장 찾기
        question_words = set(question.lower().split())
        best_sentence = ""
        best_score = 0
        
        for sentence in sentences:
            sentence_words = set(sentence.lower().split())
            score = len(question_words & sentence_words)
            if score > best_score:
                best_score = score
                best_sentence = sentence.strip()
        
        if best_sentence:
            return best_sentence + "."
        else:
            return "[컨텍스트 기반 답변 생성 필요]"
    
    def _validate_quality(self, qa_pair: Dict) -> float:
        """품질 검증 (0-100 점수)"""
        score = 0
        
        # 기본 점수
        if qa_pair.get('question'):
            score += 20
            if len(qa_pair['question']) > 20:
                score += 10
        
        if qa_pair.get('answer'):
            score += 30
            if len(qa_pair['answer']) > self.config.QUALITY_CONFIG['min_answer_length']:
                score += 20
        
        # 추가 정보
        if qa_pair.get('explanation'):
            score += 10
        
        if qa_pair.get('keywords'):
            score += 10
        
        # 폴백 사용 시 감점
        if qa_pair.get('fallback'):
            score -= 20
        
        return min(max(score, 0), 100)
    
    def get_stats(self) -> Dict:
        """통계 반환"""
        total = self.stats['total_attempts']
        if total == 0:
            return self.stats
        
        return {
            **self.stats,
            'success_rate': round(self.stats['successful'] / total * 100, 1),
            'answer_rate': round(
                self.stats['with_answer'] / max(self.stats['successful'], 1) * 100, 1
            ),
            'retry_rate': round(self.stats['retry_count'] / total * 100, 1),
            'fallback_rate': round(self.stats['fallback_used'] / total * 100, 1),
        }

print("✅ 답변 보장 생성기 클래스 정의 완료")
print("🔧 주요 기능:")
print("  - 통합형/분리형 선택 가능")
print("  - 3단계 폴백 시스템")
print("  - 품질 검증")
print("  - RAG 통합")

In [ ]:
# ========================================
# 🎓 Chain-of-Thought (CoT) 생성기 - 단계별 사고 과정
# ========================================
#
# 💡 CoT를 백엔드 관점에서 이해하기:
#   일반 API: Request → Response (바로 응답)
#   CoT API: Request → Think → Verify → Improve → Response (검증 후 응답)
#   
#   비유: 코드 리뷰 프로세스
#   1. 초안 작성 (Initial) 
#   2. 셀프 리뷰 (Self-Verification)
#   3. 수정 (Improvement)
#   4. 최종 리뷰 (Final Check)

class ChainOfThoughtGenerator:
    """
    Chain-of-Thought (CoT) 데이터 생성기
    
    🔄 4단계 검증 프로세스:
    1. 초기 생성: 첫 번째 시도
    2. 자가 검증: "이게 맞나?" 스스로 체크
    3. 개선: "이렇게 하면 더 낫겠다" 수정
    4. 최종 검증: "이제 괜찮은가?" 마지막 체크
    
    백엔드 비유: 
    - 초기 생성 = MVP 개발
    - 자가 검증 = Unit Test
    - 개선 = Refactoring  
    - 최종 검증 = Integration Test
    """
    
    def __init__(self, config: ExperimentConfig, rag_system: RAGSystem = None):
        """
        초기화
        
        💡 의존성 주입 패턴 사용
        - config: 설정 객체 (application.yml 같은)
        - rag_system: 선택적 의존성 (Optional dependency)
        """
        self.config = config
        self.cot_config = config.COT_CONFIG
        self.rag = rag_system
        
        # 모델 관련 (lazy loading)
        self.model = None
        self.tokenizer = None
        self.model_loaded = False
        
        # CoT 프롬프트 템플릿 로드
        # 💡 각 단계별로 다른 프롬프트 사용 (Strategy Pattern)
        self.cot_prompts = self._load_cot_prompts()
        
        # 캐시 설정 (동일 입력 재사용)
        # 💡 백엔드의 Redis 캐싱과 유사
        self.cache = {} if config.COT_CONFIG['cache_results'] else None
        
        # 통계 추적 (모니터링용)
        self.stats = {
            'total_attempts': 0,      # 총 시도
            'successful': 0,           # 성공
            'failed': 0,              # 실패
            'avg_iterations': 0,      # 평균 반복 횟수
            'improvement_count': 0,   # 개선 횟수
            'cache_hits': 0,          # 캐시 히트
            'quality_scores': []      # 품질 점수들
        }
    
    def generate_qa_pair(self, context: str, question_type: str = "주관식") -> Optional[Dict]:
        """
        CoT 방식으로 QA 쌍 생성 - 메인 엔트리 포인트
        
        🔄 실행 흐름:
        1. 캐시 확인 (있으면 바로 반환)
        2. 초기 생성
        3. 품질 체크 루프 (최대 N번)
           - 자가 검증
           - 점수 확인
           - 필요시 개선
        4. 최종 검증
        5. 결과 반환
        
        Args:
            context: 참고 문서 (컨텍스트)
            question_type: 문제 유형
            
        Returns:
            생성된 QA 쌍 또는 None (실패 시)
        """
        self.stats['total_attempts'] += 1
        
        # 1. 캐시 확인 (백엔드의 캐싱 레이어)
        if self.cache is not None:
            # 해시 키 생성 (context의 앞 200자로)
            cache_key = hash(f"{context[:200]}_{question_type}")
            if cache_key in self.cache:
                self.stats['cache_hits'] += 1
                if self.config.EXPERIMENT_MODE['verbose']:
                    print("  💾 캐시에서 결과 반환 (Cache Hit\!)")
                return self.cache[cache_key]
        
        try:
            # 2. RAG 활용 (선택적)
            # 💡 백엔드의 외부 API 호출과 유사
            if self.rag and self.config.RAG_CONFIG['use_rag']:
                query = f"{question_type} 문제 생성을 위한 {context[:100]}"
                retrieved = self.rag.search(query)  # DB 검색
                
                if retrieved:
                    # 검색 결과를 컨텍스트에 추가
                    additional = "\n".join([r['text'][:200] for r in retrieved[:2]])
                    context = f"{context}\n\n관련 문서:\n{additional}"
            
            # 3. 단계별 실행
            if self.config.EXPERIMENT_MODE['verbose']:
                print(f"  [CoT] {question_type} 생성 시작...")
                print("    🔄 [1/4] 초기 문제 생성...")
            
            # 3-1. 초기 생성 (첫 시도)
            initial_qa = self._initial_generation(context, question_type)
            if not initial_qa:
                raise ValueError("초기 생성 실패")
            
            current_question = initial_qa['question']
            current_answer = initial_qa['answer']
            
            # 3-2. 반복적 개선 루프
            iteration_count = 0
            for i in range(self.cot_config['max_iterations']):
                if not self.cot_config['use_self_verification']:
                    break  # 자가 검증 비활성화면 스킵
                    
                iteration_count += 1
                
                # 자가 검증 단계
                if self.config.EXPERIMENT_MODE['verbose']:
                    print(f"    🔄 [{2+i*2}/4] 자가 검증...")
                
                # 현재 QA를 검증
                verification = self._self_verification(current_question, current_answer)
                
                # 점수 추출 (1-10 → 1-100으로 변환)
                score = self._extract_score(verification)
                
                # 품질 기준 통과 체크
                if score >= self.cot_config['quality_threshold']:
                    if self.config.EXPERIMENT_MODE['verbose']:
                        print(f"      ✅ 품질 기준 통과\! (점수: {score}/100)")
                    break  # 충분히 좋으면 종료
                
                # 개선 필요
                if not self.cot_config['use_improvement']:
                    break  # 개선 비활성화면 스킵
                    
                if self.config.EXPERIMENT_MODE['verbose']:
                    print(f"    🔄 [{3+i*2}/4] 개선 생성...")
                    print(f"      (현재 점수: {score}/100)")
                
                # 피드백 기반 개선
                improved_qa = self._improvement_generation(
                    current_question, 
                    current_answer, 
                    verification  # 검증 피드백 전달
                )
                
                if improved_qa:
                    # 개선된 버전으로 교체
                    current_question = improved_qa['question']
                    current_answer = improved_qa['answer']
                    self.stats['improvement_count'] += 1
            
            # 3-3. 최종 검증
            final_score = 70  # 기본값
            if self.cot_config['use_final_check']:
                if self.config.EXPERIMENT_MODE['verbose']:
                    print("    🔄 [4/4] 최종 검증...")
                
                final_check = self._final_check(current_question, current_answer)
                final_score = self._extract_final_score(final_check)
                
                if self.config.EXPERIMENT_MODE['verbose']:
                    print(f"      최종 품질 점수: {final_score}/100")
            
            # 4. 결과 준비
            result = {
                'question': current_question,
                'answer': current_answer,
                'context': context[:500],
                'question_type': question_type,
                'generation_mode': 'cot',
                'quality_score': final_score,
                'iterations': iteration_count,  # 몇 번 개선했는지
                'model': self.config.MODEL_NAME,
                'timestamp': datetime.now().isoformat()
            }
            
            # 5. 통계 업데이트
            self.stats['successful'] += 1
            self.stats['quality_scores'].append(final_score)
            
            # 6. 캐시 저장 (품질 좋은 것만)
            if self.cache is not None and final_score >= self.cot_config['quality_threshold']:
                cache_key = hash(f"{context[:200]}_{question_type}")
                self.cache[cache_key] = result
                
            return result
            
        except Exception as e:
            logger.error(f"[CoT] 생성 오류: {e}")
            self.stats['failed'] += 1
            return None
    
    def _initial_generation(self, context: str, question_type: str) -> Optional[Dict]:
        """
        1단계: 초기 생성
        
        💡 첫 번째 시도 - MVP처럼 빠르게 만들기
        Temperature를 약간 높게(0.7) 설정해서 창의적인 문제 생성
        """
        prompt = self.cot_prompts['initial_generation'].format(
            context=context[:800],  # 너무 길면 잘라서
            question_type=question_type
        )
        
        # 초기 생성은 약간 창의적으로 (temperature 높게)
        temp = self.config.COT_GENERATION_PARAMS.get('temperature_initial', 0.7)
        generated = self.generate_text(prompt, temperature=temp)
        
        # 생성된 텍스트에서 Q&A 추출
        return self._parse_qa(generated)
    
    def _self_verification(self, question: str, answer: str) -> str:
        """
        2단계: 자가 검증
        
        💡 스스로 체크 - Code Review처럼
        Temperature를 낮게(0.3) 설정해서 비판적으로 검토
        
        체크 포인트:
        - 문제가 명확한가?
        - 답변이 정확한가?
        - 금융 용어가 올바른가?
        """
        prompt = self.cot_prompts['self_verification'].format(
            question=question,
            answer=answer
        )
        
        # 검증은 보수적으로 (temperature 낮게)
        temp = self.config.COT_GENERATION_PARAMS.get('temperature_verification', 0.3)
        verification = self.generate_text(prompt, temperature=temp, max_tokens=300)
        
        return verification
    
    def _improvement_generation(self, question: str, answer: str, feedback: str) -> Optional[Dict]:
        """
        3단계: 개선 생성
        
        💡 피드백 반영 - Refactoring처럼
        검증에서 나온 문제점을 수정
        
        개선 전략:
        - 지적된 문제점 수정
        - 더 명확한 표현 사용
        - 구체적 예시 추가
        """
        # 질문 기반 RAG 재검색 (더 나은 답변을 위해)
        enhanced_context = ""
        if self.rag and self.config.RAG_CONFIG['use_rag']:
            # 질문으로 더 정확한 문서 검색
            retrieved = self.rag.search(question, top_k=5)
            
            if retrieved:
                # 검색 결과를 추가 컨텍스트로
                enhanced_context = "\n\n### 질문 관련 참고 자료:\n" + "\n".join([
                    f"- {doc['text'][:200]}"
                    for doc in retrieved[:3]
                ])
        
        # 개선 프롬프트 생성
        prompt = f"""{self.cot_prompts['improvement']}

{enhanced_context}""".format(
            question=question,
            answer=answer,
            feedback=feedback
        )
        
        # 개선은 중간 온도로 (균형)
        temp = self.config.COT_GENERATION_PARAMS.get('temperature_improvement', 0.5)
        generated = self.generate_text(prompt, temperature=temp)
        
        return self._parse_qa(generated)
    
    def _final_check(self, question: str, answer: str) -> str:
        """
        4단계: 최종 검증
        
        💡 마지막 체크 - Production 배포 전 체크처럼
        FSKU 시험 출제 가능한 수준인지 최종 확인
        """
        prompt = self.cot_prompts['final_check'].format(
            question=question,
            answer=answer
        )
        
        # 최종 검증도 보수적으로
        temp = self.config.COT_GENERATION_PARAMS.get('temperature_final', 0.3)
        check_result = self.generate_text(prompt, temperature=temp, max_tokens=200)
        
        return check_result
    
    def get_stats(self) -> Dict:
        """
        통계 반환 - 모니터링용
        
        💡 백엔드의 메트릭 수집과 유사
        - 성공률 = API Success Rate
        - 개선률 = Cache Hit Rate  
        - 품질 점수 = Response Time
        """
        total = self.stats['total_attempts']
        if total == 0:
            return self.stats
        
        avg_quality = np.mean(self.stats['quality_scores']) if self.stats['quality_scores'] else 0
        
        return {
            **self.stats,
            'success_rate': round(self.stats['successful'] / total * 100, 1),
            'improvement_rate': round(self.stats['improvement_count'] / max(self.stats['successful'], 1) * 100, 1),
            'cache_hit_rate': round(self.stats['cache_hits'] / total * 100, 1) if self.cache else 0,
            'avg_quality_score': round(avg_quality, 1)
        }

print("✅ Chain-of-Thought (CoT) 생성기 정의 완료\!")
print("\n🎯 CoT 사용 시점:")
print("  - 일반 생성: 대량 데이터가 필요할 때 (속도 우선)")
print("  - CoT 생성: 고품질 데이터가 필요할 때 (품질 우선)")
print("\n💡 백엔드 비유:")
print("  - 일반 = Sync API (바로 응답)")
print("  - CoT = Async + Queue + Retry (신중한 처리)")

## 4-1. Chain-of-Thought (CoT) 데이터 생성기 ⭐

## 5. 비교 테스트: 통합형 vs 분리형

In [ ]:
def compare_generation_modes():
    """
    통합형과 분리형 비교 테스트
    실제로 두 모드를 실행하여 성능 비교
    """
    
    # 테스트용 컨텍스트
    test_contexts = [
        """금융보안원(FSI)은 국내 금융 IT 보안을 총괄하는 전문기관으로, 
        금융권 사이버 보안 강화를 위한 다양한 정책과 기술을 개발하고 있습니다. 
        특히 전자금융거래의 안전성 확보를 위해 다단계 인증, 암호화 기술, 
        이상거래 탐지 시스템 등을 운영하고 있으며, 금융회사들의 보안 수준을 
        정기적으로 점검하고 평가하는 역할을 수행합니다.""",
        
        """바젤III 규제는 은행의 자본 적정성을 강화하기 위한 국제 기준으로,
        보통주자본비율 4.5%, Tier1 자본비율 6%, 총자본비율 8% 이상을
        유지해야 합니다. 또한 유동성 커버리지 비율(LCR)과 순안정자금조달비율(NSFR)을
        도입하여 은행의 유동성 리스크 관리를 강화했습니다.""",
    ]
    
    print("="*80)
    print("📊 통합형 vs 분리형 비교 테스트")
    print("="*80)
    
    results = {}
    
    # 1. 통합형 테스트
    print("\n1️⃣ 통합형 모드 테스트")
    print("-"*40)
    
    # 통합형 설정
    integrated_config = ExperimentConfig()
    integrated_config.GENERATION_MODE = "integrated"
    integrated_config.EXPERIMENT_MODE['verbose'] = True
    
    integrated_gen = AnswerGuaranteedGenerator(integrated_config)
    
    integrated_results = []
    integrated_times = []
    
    for i, context in enumerate(test_contexts):
        for question_type in ["객관식", "주관식", "단답형"]:
            print(f"\n테스트 {i+1} - {question_type}:")
            
            start_time = time.time()
            result = integrated_gen.generate_qa_pair(context, question_type)
            elapsed = time.time() - start_time
            
            if result:
                integrated_results.append(result)
                integrated_times.append(elapsed)
                
                print(f"  ✅ 성공 ({elapsed:.2f}초)")
                print(f"  답변 길이: {len(result.get('answer', ''))} 글자")
                print(f"  품질 점수: {result.get('quality_score', 0):.0f}/100")
            else:
                print(f"  ❌ 실패")
    
    results['integrated'] = {
        'data': integrated_results,
        'times': integrated_times,
        'stats': integrated_gen.get_stats()
    }
    
    # 2. 분리형 테스트
    print("\n\n2️⃣ 분리형 모드 테스트")
    print("-"*40)
    
    # 분리형 설정
    separated_config = ExperimentConfig()
    separated_config.GENERATION_MODE = "separated"
    separated_config.EXPERIMENT_MODE['verbose'] = True
    
    separated_gen = AnswerGuaranteedGenerator(separated_config)
    
    separated_results = []
    separated_times = []
    
    for i, context in enumerate(test_contexts):
        for question_type in ["객관식", "주관식", "단답형"]:
            print(f"\n테스트 {i+1} - {question_type}:")
            
            start_time = time.time()
            result = separated_gen.generate_qa_pair(context, question_type)
            elapsed = time.time() - start_time
            
            if result:
                separated_results.append(result)
                separated_times.append(elapsed)
                
                print(f"  ✅ 성공 ({elapsed:.2f}초)")
                print(f"  답변 길이: {len(result.get('answer', ''))} 글자")
                print(f"  품질 점수: {result.get('quality_score', 0):.0f}/100")
                print(f"  폴백 사용: {result.get('fallback', False)}")
            else:
                print(f"  ❌ 실패")
    
    results['separated'] = {
        'data': separated_results,
        'times': separated_times,
        'stats': separated_gen.get_stats()
    }
    
    # 3. 결과 비교
    print("\n" + "="*80)
    print("📈 비교 결과 요약")
    print("="*80)
    
    # 통계 비교 테이블
    comparison_data = []
    
    for mode in ['integrated', 'separated']:
        mode_stats = results[mode]['stats']
        mode_times = results[mode]['times']
        mode_data = results[mode]['data']
        
        avg_time = np.mean(mode_times) if mode_times else 0
        avg_answer_len = np.mean([len(d.get('answer', '')) for d in mode_data]) if mode_data else 0
        avg_quality = np.mean([d.get('quality_score', 0) for d in mode_data]) if mode_data else 0
        
        comparison_data.append({
            '모드': mode.upper(),
            '성공률': f"{mode_stats.get('success_rate', 0):.1f}%",
            '답변 포함률': f"{mode_stats.get('answer_rate', 0):.1f}%",
            '평균 시간': f"{avg_time:.2f}초",
            '평균 답변 길이': f"{avg_answer_len:.0f}자",
            '평균 품질': f"{avg_quality:.0f}/100",
            '재시도율': f"{mode_stats.get('retry_rate', 0):.1f}%",
            '폴백 사용률': f"{mode_stats.get('fallback_rate', 0):.1f}%"
        })
    
    # 테이블 출력
    df_comparison = pd.DataFrame(comparison_data)
    print("\n비교 테이블:")
    print(df_comparison.to_string(index=False))
    
    # 4. 샘플 출력
    print("\n" + "="*80)
    print("📝 생성된 샘플 비교")
    print("="*80)
    
    for mode in ['integrated', 'separated']:
        print(f"\n[{mode.upper()} 모드 샘플]")
        print("-"*40)
        
        if results[mode]['data']:
            sample = results[mode]['data'][0]
            print(f"문제 유형: {sample.get('question_type')}")
            print(f"문제: {sample.get('question', 'N/A')[:150]}...")
            print(f"답변: {sample.get('answer', 'N/A')[:150]}...")
            print(f"품질: {sample.get('quality_score', 0):.0f}/100")
    
    # 5. 권장사항
    print("\n" + "="*80)
    print("💡 권장사항")
    print("="*80)
    
    # 속도 비교
    integrated_avg_time = np.mean(results['integrated']['times']) if results['integrated']['times'] else 999
    separated_avg_time = np.mean(results['separated']['times']) if results['separated']['times'] else 999
    
    if integrated_avg_time < separated_avg_time * 0.7:
        print("🚀 속도 우선: 통합형 모드 추천 (약 {:.0f}% 빠름)".format(
            (1 - integrated_avg_time/separated_avg_time) * 100
        ))
    elif separated_avg_time < integrated_avg_time * 0.7:
        print("🚀 속도 우선: 분리형 모드 추천 (약 {:.0f}% 빠름)".format(
            (1 - separated_avg_time/integrated_avg_time) * 100
        ))
    else:
        print("⚖️ 속도: 두 모드 비슷함")
    
    # 품질 비교
    integrated_avg_quality = np.mean([d.get('quality_score', 0) for d in results['integrated']['data']])
    separated_avg_quality = np.mean([d.get('quality_score', 0) for d in results['separated']['data']])
    
    if integrated_avg_quality > separated_avg_quality + 5:
        print("⭐ 품질 우선: 통합형 모드 추천 (평균 {:.0f}점 높음)".format(
            integrated_avg_quality - separated_avg_quality
        ))
    elif separated_avg_quality > integrated_avg_quality + 5:
        print("⭐ 품질 우선: 분리형 모드 추천 (평균 {:.0f}점 높음)".format(
            separated_avg_quality - integrated_avg_quality
        ))
    else:
        print("⚖️ 품질: 두 모드 비슷함")
    
    print("\n📌 일반적 권장사항:")
    print("  - 대량 생성 시: 통합형 (빠름)")
    print("  - 고품질 필요 시: 분리형 (정확함)")
    print("  - 균형: 통합형 + 품질 검증")
    
    return results

# 테스트 실행 (주석 해제하여 실행)
# comparison_results = compare_generation_modes()

## 6. 대량 데이터 생성 및 실행

In [ ]:
def run_bulk_generation():
    """
    대량 데이터 생성 실행
    설정에 따라 통합형 또는 분리형으로 실행
    """
    
    print("="*80)
    print("🚀 FSKU 대량 데이터 생성 시작")
    print("="*80)
    
    # 설정 출력
    print("\n📋 현재 설정:")
    print(f"  - 모델: {config.MODEL_NAME}")
    print(f"  - 생성 모드: {config.GENERATION_MODE}")
    print(f"  - 목표 개수: {config.BATCH_CONFIG['target_count']}개")
    print(f"  - Temperature: {config.GENERATION_PARAMS['temperature']}")
    print(f"  - 품질 임계값: {config.QUALITY_CONFIG['quality_threshold']}")
    print(f"  - RAG 사용: {config.RAG_CONFIG['use_rag']}")
    
    # 1. RAG 시스템 초기화 (선택적)
    rag_system = None
    if config.RAG_CONFIG['use_rag']:
        print("\n📚 RAG 시스템 초기화...")
        rag_system = RAGSystem(config)
        rag_system.initialize()
    
    # 2. 생성기 초기화
    print("\n🤖 생성기 초기화...")
    generator = AnswerGuaranteedGenerator(config, rag_system)
    generator.load_model()
    
    # 3. 샘플 컨텍스트 준비
    sample_contexts = [
        """전자금융거래법은 전자금융거래의 안전성과 신뢰성을 확보하기 위한 법률로,
        금융회사는 전자금융거래 시 다단계 인증을 적용해야 하며,
        거래 내역은 최소 5년간 보관해야 합니다.""",
        
        """개인정보보호법에 따라 금융회사는 고객의 개인정보를 수집할 때
        명확한 동의를 받아야 하며, 수집 목적 외 사용은 금지됩니다.
        개인정보 유출 시 24시간 내 신고 의무가 있습니다.""",
        
        """바젤III 규제는 은행의 자본 적정성을 강화하기 위한 국제 기준으로,
        보통주자본비율 4.5%, Tier1 자본비율 6%, 총자본비율 8% 이상을
        유지해야 합니다.""",
        
        """금융실명거래법은 금융거래의 투명성을 확보하기 위한 법률로,
        모든 금융거래는 실명으로 이루어져야 하며,
        차명거래는 엄격히 금지됩니다.""",
        
        """자금세탁방지제도(AML)는 불법자금의 세탁을 방지하기 위한 제도로,
        금융회사는 고객확인의무(CDD)를 수행해야 하며,
        의심거래는 FIU에 보고해야 합니다.""",
    ]
    
    # 4. 대량 생성
    print(f"\n📊 데이터 생성 시작 (목표: {config.BATCH_CONFIG['target_count']}개)")
    print("-"*40)
    
    generated_data = []
    attempts = 0
    max_attempts = config.BATCH_CONFIG['target_count'] * config.BATCH_CONFIG['max_attempts_ratio']
    
    # 문제 유형별 목표
    type_targets = {
        qtype: int(config.BATCH_CONFIG['target_count'] * ratio)
        for qtype, ratio in config.QUESTION_TYPE_DISTRIBUTION.items()
    }
    type_counts = {qtype: 0 for qtype in type_targets}
    
    # 진행 상황 표시 간격
    progress_interval = max(config.BATCH_CONFIG['target_count'] // 10, 1)
    save_interval = config.BATCH_CONFIG['save_interval']
    
    # 시작 시간
    start_time = time.time()
    
    while len(generated_data) < config.BATCH_CONFIG['target_count'] and attempts < max_attempts:
        # 컨텍스트 선택
        context = np.random.choice(sample_contexts)
        
        # 문제 유형 선택 (부족한 유형 우선)
        remaining_types = [
            qtype for qtype, target in type_targets.items()
            if type_counts[qtype] < target
        ]
        
        if remaining_types:
            question_type = np.random.choice(remaining_types)
        else:
            question_type = np.random.choice(list(config.QUESTION_TYPE_DISTRIBUTION.keys()))
        
        # 생성
        qa_pair = generator.generate_qa_pair(context, question_type)
        
        # 검증
        if qa_pair and qa_pair.get('answer'):
            # 품질 확인
            if qa_pair.get('quality_score', 0) >= config.QUALITY_CONFIG['quality_threshold']:
                generated_data.append(qa_pair)
                type_counts[question_type] += 1
                
                # 진행 상황 출력
                if len(generated_data) % progress_interval == 0:
                    elapsed = time.time() - start_time
                    rate = len(generated_data) / elapsed
                    eta = (config.BATCH_CONFIG['target_count'] - len(generated_data)) / rate
                    
                    print(f"  진행: {len(generated_data)}/{config.BATCH_CONFIG['target_count']} "
                          f"({len(generated_data)/config.BATCH_CONFIG['target_count']*100:.1f}%) "
                          f"| 속도: {rate:.1f}개/초 | 예상 시간: {eta:.0f}초")
                
                # 중간 저장
                if len(generated_data) % save_interval == 0:
                    temp_path = AUGMENTED_DIR / f"temp_batch_{len(generated_data)}.json"
                    with open(temp_path, 'w', encoding='utf-8') as f:
                        json.dump(generated_data, f, ensure_ascii=False, indent=2)
                    print(f"  💾 중간 저장: {temp_path}")
        
        attempts += 1
    
    # 생성 완료
    elapsed_total = time.time() - start_time
    
    print("\n" + "="*80)
    print("✅ 생성 완료!")
    print("="*80)
    
    # 통계 출력
    print(f"\n📊 생성 통계:")
    print(f"  - 총 생성: {len(generated_data)}개")
    print(f"  - 총 시도: {attempts}회")
    print(f"  - 성공률: {len(generated_data)/attempts*100:.1f}%")
    print(f"  - 소요 시간: {elapsed_total:.1f}초")
    print(f"  - 평균 속도: {len(generated_data)/elapsed_total:.2f}개/초")
    
    print("\n문제 유형별 분포:")
    for qtype, count in type_counts.items():
        if count > 0:
            print(f"  - {qtype}: {count}개 ({count/len(generated_data)*100:.1f}%)")
    
    # 답변 통계
    answer_lengths = [len(d['answer']) for d in generated_data]
    quality_scores = [d.get('quality_score', 0) for d in generated_data]
    
    if answer_lengths:
        print(f"\n답변 길이 통계:")
        print(f"  - 평균: {np.mean(answer_lengths):.1f} 글자")
        print(f"  - 최소: {np.min(answer_lengths)} 글자")
        print(f"  - 최대: {np.max(answer_lengths)} 글자")
    
    if quality_scores:
        print(f"\n품질 점수 통계:")
        print(f"  - 평균: {np.mean(quality_scores):.1f}/100")
        print(f"  - 최소: {np.min(quality_scores):.0f}/100")
        print(f"  - 최대: {np.max(quality_scores):.0f}/100")
    
    # 생성기 통계
    gen_stats = generator.get_stats()
    print(f"\n생성기 통계:")
    print(f"  - 답변 포함률: {gen_stats['answer_rate']:.1f}%")
    print(f"  - 재시도율: {gen_stats['retry_rate']:.1f}%")
    print(f"  - 폴백 사용률: {gen_stats['fallback_rate']:.1f}%")
    
    # 5. 데이터 저장
    if generated_data:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # JSON 저장
        json_path = AUGMENTED_DIR / f"fsku_data_{config.GENERATION_MODE}_{timestamp}.json"
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(generated_data, f, ensure_ascii=False, indent=2)
        
        # JSONL 저장
        jsonl_path = AUGMENTED_DIR / f"fsku_data_{config.GENERATION_MODE}_{timestamp}.jsonl"
        with open(jsonl_path, 'w', encoding='utf-8') as f:
            for item in generated_data:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')
        
        print(f"\n💾 데이터 저장 완료:")
        print(f"  - JSON: {json_path}")
        print(f"  - JSONL: {jsonl_path}")
        print(f"  - 총 {len(generated_data)}개 항목")
        
        # 통계 저장
        if config.EXPERIMENT_MODE['save_stats']:
            stats_path = AUGMENTED_DIR / f"stats_{config.GENERATION_MODE}_{timestamp}.json"
            stats_data = {
                'config': {
                    'model': config.MODEL_NAME,
                    'mode': config.GENERATION_MODE,
                    'temperature': config.GENERATION_PARAMS['temperature'],
                    'quality_threshold': config.QUALITY_CONFIG['quality_threshold'],
                },
                'results': {
                    'total_generated': len(generated_data),
                    'total_attempts': attempts,
                    'success_rate': len(generated_data)/attempts*100,
                    'elapsed_time': elapsed_total,
                    'type_distribution': type_counts,
                    'answer_length_mean': np.mean(answer_lengths),
                    'quality_score_mean': np.mean(quality_scores),
                },
                'generator_stats': gen_stats,
                'timestamp': timestamp
            }
            
            with open(stats_path, 'w', encoding='utf-8') as f:
                json.dump(stats_data, f, ensure_ascii=False, indent=2)
            
            print(f"  - 통계: {stats_path}")
    
    print("\n✨ 모든 작업 완료!")
    return generated_data

# 실행 (주석 해제하여 실행)
# data = run_bulk_generation()

## 7. 🧪 실험 실행 섹션

In [ ]:
# ========================================
# 🧪 실험 실행
# 아래 코드의 주석을 해제하여 실행하세요
# ========================================

print("🧪 실험 옵션:")
print("1. 단일 테스트: test_single_generation()")
print("2. 모드 비교: compare_generation_modes()")
print("3. 대량 생성: run_bulk_generation()")
print("\n원하는 실험의 주석을 해제하고 실행하세요!")

# 1. 단일 생성 테스트 (빠른 테스트)
def test_single_generation():
    """단일 QA 생성 테스트"""
    print("\n🔬 단일 생성 테스트")
    print("-"*40)
    
    # 테스트 컨텍스트
    test_context = """금융보안원은 금융 IT 보안을 담당하는 기관으로,
    전자금융거래의 안전성 확보를 위해 다양한 보안 기술을 운영합니다."""
    
    # 생성기 초기화
    generator = AnswerGuaranteedGenerator(config)
    generator.load_model()
    
    # 생성
    result = generator.generate_qa_pair(test_context, "주관식")
    
    if result:
        print(f"\n✅ 생성 성공!")
        print(f"문제: {result['question']}")
        print(f"답변: {result['answer']}")
        print(f"품질: {result.get('quality_score', 0):.0f}/100")
        print(f"모드: {result['generation_mode']}")
    else:
        print("❌ 생성 실패")
    
    return result

# test_single_generation()

# 2. 통합형 vs 분리형 비교
# compare_generation_modes()

# 3. 대량 생성 실행
# data = run_bulk_generation()

In [ ]:
# ========================================
# 🚀 개선된 RAG 활용 답변 생성기
# ========================================

class ImprovedAnswerGenerator:
    """
    개선된 답변 생성기
    - 생성된 질문을 기반으로 RAG 재검색
    - 더 정확한 답변 생성
    """
    
    def __init__(self, config: ExperimentConfig, rag_system: RAGSystem = None):
        self.config = config
        self.rag = rag_system
        
    def generate_answer_with_rag(self, question: str, original_context: str = None) -> str:
        """
        질문 기반 RAG 검색 후 답변 생성
        
        Args:
            question: 생성된 질문
            original_context: 원본 컨텍스트 (선택적)
            
        Returns:
            생성된 답변
        """
        # 1. 질문으로 RAG 검색
        if self.rag:
            print(f"  🔍 질문 기반 RAG 재검색...")
            retrieved_docs = self.rag.search(question, top_k=5)
            
            # 검색된 문서 결합
            rag_context = "\n\n".join([
                f"[참고 {i+1}] {doc['text'][:300]}"
                for i, doc in enumerate(retrieved_docs[:3])
            ])
            
            # 원본 컨텍스트와 결합
            if original_context:
                full_context = f"""원본 문서:
{original_context[:500]}

관련 참고 자료:
{rag_context}"""
            else:
                full_context = rag_context
                
        else:
            full_context = original_context or ""
            
        # 2. 답변 생성 프롬프트
        answer_prompt = f"""당신은 금융 전문가입니다. 다음 질문에 대해 제공된 참고 자료를 활용하여 정확하고 상세한 답변을 작성하세요.

### 참고 자료:
{full_context}

### 질문:
{question}

### 답변 작성 지침:
1. 참고 자료의 정보를 정확히 인용하세요
2. 금융 전문 용어를 정확히 사용하세요
3. 구체적인 수치나 규정이 있다면 반드시 포함하세요
4. 답변은 완전한 문장으로 작성하세요
5. 핵심 내용을 먼저 제시하고 부가 설명을 추가하세요

### 답변:"""
        
        return answer_prompt
    
    def separated_generation_improved(self, context: str, question_type: str) -> Dict:
        """
        개선된 분리형 생성 (RAG 활용 강화)
        """
        # 1단계: 질문 생성 (기존과 동일)
        question_prompt = self.get_question_prompt(context, question_type)
        question = self.generate_text(question_prompt)
        
        # 2단계: 질문 기반 RAG 검색 후 답변 생성
        answer_prompt = self.generate_answer_with_rag(question, context)
        answer = self.generate_text(answer_prompt)
        
        return {
            'question': question,
            'answer': answer,
            'mode': 'separated_improved',
            'rag_used': self.rag is not None
        }

# AnswerGuaranteedGenerator 클래스 개선
class AnswerGuaranteedGeneratorV2(AnswerGuaranteedGenerator):
    """
    개선된 답변 보장 생성기 (RAG 활용 강화)
    """
    
    def generate_qa_pair(self, context: str, question_type: str = "주관식") -> Optional[Dict]:
        """
        QA 쌍 생성 (개선된 버전)
        """
        self.stats['total_attempts'] += 1
        
        try:
            if self.generation_mode == "integrated":
                # 통합형: 기존과 동일
                return self._generate_integrated(context, question_type)
                
            elif self.generation_mode == "separated":
                # 분리형: RAG 개선 적용
                return self._generate_separated_improved(context, question_type)
                
            else:
                raise ValueError(f"지원하지 않는 모드: {self.generation_mode}")
                
        except Exception as e:
            logger.error(f"QA 생성 오류: {e}")
            self.stats['failed'] += 1
            return None
    
    def _generate_separated_improved(self, context: str, question_type: str) -> Optional[Dict]:
        """
        개선된 분리형 생성 (질문 기반 RAG 재검색)
        """
        if self.config.EXPERIMENT_MODE['verbose']:
            print(f"  [분리형-개선] {question_type} 생성...")
            
        # 1단계: 질문 생성
        if self.config.EXPERIMENT_MODE['verbose']:
            print("    📝 질문 생성 중...")
            
        question_prompt = self.prompts.get_separated_question_prompt(context, question_type)
        question = self.generate_text(question_prompt, max_tokens=150)
        
        if not question or len(question.strip()) < 10:
            return None
            
        # 2단계: 질문 기반 RAG 재검색
        if self.rag and self.config.RAG_CONFIG['use_rag']:
            if self.config.EXPERIMENT_MODE['verbose']:
                print(f"    🔍 질문 기반 RAG 재검색...")
                
            # 질문으로 관련 문서 검색
            retrieved_docs = self.rag.search(question, top_k=5)
            
            # 검색된 문서를 컨텍스트에 추가
            rag_context = "\n\n".join([
                f"[참고 {i+1}] {doc['text'][:400]}"
                for i, doc in enumerate(retrieved_docs[:3])
            ])
            
            # 원본 컨텍스트와 결합
            enhanced_context = f"""### 원본 문서:
{context[:600]}

### 질문 관련 추가 참고 자료:
{rag_context}"""
        else:
            enhanced_context = context
            
        # 3단계: 강화된 컨텍스트로 답변 생성
        if self.config.EXPERIMENT_MODE['verbose']:
            print("    💡 답변 생성 중 (RAG 강화)...")
            
        answer_prompt = f"""당신은 한국 금융감독원의 FSKU 시험 출제위원입니다.
다음 질문에 대해 제공된 참고 자료를 활용하여 정확하고 완전한 답변을 작성하세요.

{enhanced_context}

### 질문:
{question}

### 답변 작성 지침:
1. 제공된 참고 자료의 정보를 정확히 활용하세요
2. 금융 전문 용어와 수치를 정확히 포함하세요
3. FSKU 시험 답안 수준의 완성도를 갖추세요
4. 핵심 내용을 먼저, 부가 설명을 나중에 제시하세요
5. 답변은 완전한 문장으로 작성하세요

### 모범 답안:"""
        
        answer = self.generate_text(answer_prompt, max_tokens=300)
        
        # 답변 검증
        if not answer or len(answer.strip()) < self.config.QUALITY_CONFIG['min_answer_length']:
            # 폴백: 간단한 프롬프트로 재시도
            if self.config.EXPERIMENT_MODE['verbose']:
                print("    ⚠️ 답변 부족, 폴백 시도...")
                
            fallback_prompt = f"질문: {question}\n정답:"
            answer = self.generate_text(fallback_prompt, max_tokens=200)
            self.stats['fallback_used'] += 1
            
        # 결과 생성
        result = {
            'question': question.strip(),
            'answer': answer.strip(),
            'context': context[:500],
            'question_type': question_type,
            'generation_mode': 'separated_improved',
            'rag_enhanced': self.rag is not None,
            'quality_score': self._calculate_quality_score(question, answer),
            'model': self.config.MODEL_NAME,
            'timestamp': datetime.now().isoformat()
        }
        
        self.stats['successful'] += 1
        self.stats['mode_stats']['separated'] += 1
        if answer and len(answer.strip()) > 10:
            self.stats['with_answer'] += 1
        else:
            self.stats['without_answer'] += 1
            
        return result

# ChainOfThoughtGenerator 개선
class ChainOfThoughtGeneratorV2(ChainOfThoughtGenerator):
    """
    개선된 CoT 생성기 (RAG 활용 강화)
    """
    
    def _initial_generation(self, context: str, question_type: str) -> Optional[Dict]:
        """
        1단계: 초기 생성 (RAG 활용)
        """
        # RAG로 컨텍스트 보강
        if self.rag and self.config.RAG_CONFIG['use_rag']:
            # 주제 추출을 위한 간단한 검색
            query = f"{question_type} 문제 생성을 위한 {context[:100]}"
            retrieved = self.rag.search(query, top_k=3)
            
            if retrieved:
                additional = "\n\n".join([
                    f"[참고] {doc['text'][:300]}"
                    for doc in retrieved[:2]
                ])
                enhanced_context = f"{context}\n\n### 추가 참고 자료:\n{additional}"
            else:
                enhanced_context = context
        else:
            enhanced_context = context
            
        prompt = self.cot_prompts['initial_generation'].format(
            context=enhanced_context[:1000],
            question_type=question_type
        )
        
        temp = self.config.COT_GENERATION_PARAMS.get('temperature_initial', 0.7)
        generated = self.generate_text(prompt, temperature=temp)
        
        return self._parse_qa(generated)
    
    def _improvement_generation(self, question: str, answer: str, feedback: str) -> Optional[Dict]:
        """
        3단계: 개선 생성 (질문 기반 RAG 재검색)
        """
        # 생성된 질문으로 RAG 재검색
        enhanced_context = ""
        if self.rag and self.config.RAG_CONFIG['use_rag']:
            # 질문으로 더 정확한 문서 검색
            retrieved = self.rag.search(question, top_k=5)
            
            if retrieved:
                enhanced_context = "\n\n### 질문 관련 참고 자료:\n" + "\n".join([
                    f"- {doc['text'][:200]}"
                    for doc in retrieved[:3]
                ])
        
        prompt = f"""{self.cot_prompts['improvement']}

{enhanced_context}""".format(
            question=question,
            answer=answer,
            feedback=feedback
        )
        
        temp = self.config.COT_GENERATION_PARAMS.get('temperature_improvement', 0.5)
        generated = self.generate_text(prompt, temperature=temp)
        
        return self._parse_qa(generated)

print("✅ RAG 활용 개선 완료\!")
print("\n🚀 개선된 기능:")
print("  1. 분리형: 질문 생성 → 질문으로 RAG 재검색 → 답변 생성")
print("  2. CoT: 각 단계에서 RAG 활용 강화")
print("  3. 답변 생성 시 질문 관련 문서 우선 참조")
print("\n💡 사용법:")
print("  - config.GENERATION_MODE = 'separated' 설정 후 실행")
print("  - 자동으로 개선된 RAG 활용 적용됨")

In [ ]:
# ========================================
# 🧪 CoT 통합 테스트 및 모드 비교
# ========================================

def test_cot_generation():
    """
    CoT 모드 단일 테스트
    """
    print("="*80)
    print("🧠 Chain-of-Thought (CoT) 테스트")
    print("="*80)
    
    # CoT 설정
    test_config = ExperimentConfig()
    test_config.GENERATION_MODE = "cot"
    test_config.COT_CONFIG['use_cot'] = True
    test_config.EXPERIMENT_MODE['verbose'] = True
    
    # RAG 시스템 초기화 (선택적)
    rag = None
    if test_config.RAG_CONFIG['use_rag']:
        print("\n📚 RAG 시스템 초기화...")
        rag = RAGSystem(test_config)
        rag.initialize()
    
    # CoT 생성기 생성
    print("\n🧠 CoT 생성기 초기화...")
    cot_gen = ChainOfThoughtGenerator(test_config, rag)
    cot_gen.load_model()
    
    # 테스트 컨텍스트
    test_context = """
    바젤III 규제는 글로벌 금융위기 이후 은행의 자본 적정성과 유동성 관리를 강화하기 위해 도입되었습니다.
    주요 내용으로는 보통주자본비율 4.5%, Tier1 자본비율 6%, 총자본비율 8% 이상 유지,
    자본보전완충자본 2.5% 추가, 경기대응완충자본 0~2.5% 탄력 운영 등이 있습니다.
    또한 레버리지 비율 3% 이상, 유동성 커버리지 비율(LCR) 100% 이상,
    순안정자금조달비율(NSFR) 100% 이상을 요구합니다.
    """
    
    print("\n🔬 CoT 4단계 프로세스 시작...")
    print("  1️⃣ 초기 생성")
    print("  2️⃣ 자가 검증")
    print("  3️⃣ 개선 생성")
    print("  4️⃣ 최종 검증")
    
    # CoT 생성 실행
    result = cot_gen.generate_qa_pair(test_context, "주관식")
    
    if result:
        print("\n✅ CoT 생성 성공\!")
        print(f"\n📝 생성된 문제:")
        print(f"  {result['question'][:200]}...")
        print(f"\n💡 생성된 답변:")
        print(f"  {result['answer'][:200]}...")
        print(f"\n📊 품질 점수: {result.get('quality_score', 0)}/100")
        print(f"🔄 개선 반복 횟수: {result.get('iterations', 0)}회")
        
        # 통계 출력
        stats = cot_gen.get_stats()
        print(f"\n📈 CoT 통계:")
        print(f"  - 성공률: {stats.get('success_rate', 0):.1f}%")
        print(f"  - 개선률: {stats.get('improvement_rate', 0):.1f}%")
        print(f"  - 평균 품질: {stats.get('avg_quality_score', 0):.1f}")
        print(f"  - 캐시 적중률: {stats.get('cache_hit_rate', 0):.1f}%")
    else:
        print("\n❌ CoT 생성 실패")
    
    return result

def compare_all_modes():
    """
    통합형 vs 분리형 vs CoT 전체 비교
    """
    print("="*80)
    print("🔬 전체 모드 비교 테스트")
    print("="*80)
    
    # 테스트 설정
    test_contexts = [
        """금융보안원(FSI)은 국내 금융 IT 보안을 총괄하는 전문기관으로,
        금융권 사이버 보안 강화를 위한 다양한 정책과 기술을 개발합니다.""",
        
        """파생상품은 기초자산의 가격 변동에 따라 가치가 결정되는 금융상품으로,
        선물, 옵션, 스왑 등이 대표적입니다. 위험 헤지와 투기 목적으로 활용됩니다."""
    ]
    
    modes = ["integrated", "separated", "cot"]
    results = {mode: [] for mode in modes}
    
    for mode in modes:
        print(f"\n{'='*40}")
        print(f"📋 {mode.upper()} 모드 테스트")
        print(f"{'='*40}")
        
        # 설정 생성
        config = ExperimentConfig()
        config.GENERATION_MODE = mode
        if mode == "cot":
            config.COT_CONFIG['use_cot'] = True
            config.CURRENT_COT_PRESET = "balanced"
        
        # 생성기 생성
        if mode == "cot":
            generator = ChainOfThoughtGenerator(config)
        else:
            generator = AnswerGuaranteedGenerator(config)
        
        generator.load_model()
        
        # 각 컨텍스트에 대해 테스트
        for i, context in enumerate(test_contexts):
            print(f"\n테스트 {i+1}:")
            start_time = time.time()
            
            result = generator.generate_qa_pair(context, "주관식")
            elapsed = time.time() - start_time
            
            if result:
                results[mode].append({
                    'result': result,
                    'time': elapsed,
                    'quality': result.get('quality_score', 70)
                })
                print(f"  ✅ 성공 ({elapsed:.2f}초)")
                print(f"  품질: {result.get('quality_score', 70)}/100")
            else:
                print(f"  ❌ 실패")
    
    # 결과 비교
    print(f"\n{'='*80}")
    print("📊 비교 결과")
    print(f"{'='*80}")
    
    for mode in modes:
        if results[mode]:
            avg_time = np.mean([r['time'] for r in results[mode]])
            avg_quality = np.mean([r['quality'] for r in results[mode]])
            success_rate = len(results[mode]) / len(test_contexts) * 100
            
            print(f"\n{mode.upper()} 모드:")
            print(f"  - 성공률: {success_rate:.0f}%")
            print(f"  - 평균 시간: {avg_time:.2f}초")
            print(f"  - 평균 품질: {avg_quality:.1f}/100")
            
            # 모드별 특징
            if mode == "integrated":
                print("  - 특징: 빠르고 일관성 있는 생성")
            elif mode == "separated":
                print("  - 특징: 정확한 답변, 느린 속도")
            elif mode == "cot":
                print("  - 특징: 최고 품질, 4단계 검증")
    
    return results

def run_cot_experiments():
    """
    CoT 실험 파라미터 테스트
    """
    print("="*80)
    print("🔬 CoT 실험 파라미터 테스트")
    print("="*80)
    
    # 테스트할 프리셋들
    presets = ["fast", "balanced", "quality"]
    preset_results = {}
    
    test_context = """
    중앙은행의 통화정책은 경제 안정화를 위한 핵심 도구입니다.
    금리 조절, 공개시장조작, 지급준비율 조정 등을 통해
    물가 안정과 완전 고용을 추구합니다.
    """
    
    for preset in presets:
        print(f"\n🧪 프리셋 테스트: {preset}")
        print("-"*40)
        
        # 설정
        config = ExperimentConfig()
        config.GENERATION_MODE = "cot"
        config.CURRENT_COT_PRESET = preset
        
        # 프리셋 적용
        if preset in config.COT_PRESETS:
            for key, value in config.COT_PRESETS[preset].items():
                if key in config.COT_CONFIG:
                    config.COT_CONFIG[key] = value
        
        print(f"  설정:")
        print(f"    - 최대 반복: {config.COT_CONFIG['max_iterations']}")
        print(f"    - 품질 임계값: {config.COT_CONFIG['quality_threshold']}")
        print(f"    - 개선 사용: {config.COT_CONFIG['use_improvement']}")
        
        # 생성기 생성
        cot_gen = ChainOfThoughtGenerator(config)
        cot_gen.load_model()
        
        # 테스트 실행
        start_time = time.time()
        result = cot_gen.generate_qa_pair(test_context, "서술형")
        elapsed = time.time() - start_time
        
        if result:
            preset_results[preset] = {
                'time': elapsed,
                'quality': result.get('quality_score', 0),
                'iterations': result.get('iterations', 0)
            }
            
            print(f"\n  결과:")
            print(f"    - 시간: {elapsed:.2f}초")
            print(f"    - 품질: {result.get('quality_score', 0)}/100")
            print(f"    - 반복: {result.get('iterations', 0)}회")
    
    # 프리셋 비교
    print(f"\n{'='*80}")
    print("📊 프리셋 비교 결과")
    print(f"{'='*80}")
    
    print(f"\n{'프리셋':<12} {'시간(초)':<10} {'품질':<10} {'반복':<10}")
    print("-"*42)
    for preset, data in preset_results.items():
        print(f"{preset:<12} {data['time']:<10.2f} {data['quality']:<10.1f} {data['iterations']:<10}")
    
    # 권장사항
    print(f"\n💡 권장사항:")
    print("  - 빠른 프로토타이핑: 'fast' 프리셋")
    print("  - 일반 사용: 'balanced' 프리셋")
    print("  - 최고 품질: 'quality' 프리셋")
    print("  - 연구/논문: 'research' 프리셋 (별도 설정)")
    
    return preset_results

# 실행 메뉴
def show_cot_menu():
    """
    CoT 실험 메뉴
    """
    print("\n" + "="*80)
    print("🧠 Chain-of-Thought (CoT) 실험 메뉴")
    print("="*80)
    print("\n실행할 실험을 선택하세요:")
    print("\n1. test_cot_generation()")
    print("   - CoT 단일 테스트")
    print("\n2. compare_all_modes()")
    print("   - 통합형 vs 분리형 vs CoT 비교")
    print("\n3. run_cot_experiments()")
    print("   - CoT 프리셋 실험")
    print("\n4. run_bulk_generation()")
    print("   - 대량 데이터 생성 (기존 함수)")
    print("\n예시: test_cot_generation()")

show_cot_menu()

## 8. 📝 사용 가이드 및 팁

### 실험 가능한 설정들

1. **생성 모드 변경**
```python
config.GENERATION_MODE = "integrated"  # 또는 "separated"
```

2. **Temperature 조정** (창의성)
```python
config.GENERATION_PARAMS['temperature'] = 0.3  # 보수적
config.GENERATION_PARAMS['temperature'] = 0.8  # 균형
config.GENERATION_PARAMS['temperature'] = 1.0  # 창의적
```

3. **품질 임계값 변경**
```python
config.QUALITY_CONFIG['quality_threshold'] = 50  # 낮은 기준
config.QUALITY_CONFIG['quality_threshold'] = 70  # 중간
config.QUALITY_CONFIG['quality_threshold'] = 90  # 높은 기준
```

4. **모델 변경**
```python
config.MODEL_NAME = "upstage/SOLAR-10.7B-v1.0"  # 더 큰 모델
```

5. **RAG 활성화/비활성화**
```python
config.RAG_CONFIG['use_rag'] = False  # RAG 없이
```

### 성능 최적화 팁

1. **속도 우선**
   - 통합형 모드 사용
   - Temperature 낮게 (0.3~0.5)
   - max_new_tokens 줄이기 (200~300)
   - RAG 비활성화

2. **품질 우선**
   - 분리형 모드 사용
   - Temperature 중간 (0.7~0.8)
   - 품질 임계값 높게 (80+)
   - RAG 활성화

3. **메모리 절약**
   - 양자화 활성화
   - 배치 크기 줄이기
   - 작은 모델 사용

### 문제 해결

1. **답변이 생성되지 않을 때**
   - 분리형 모드 시도
   - Temperature 높이기
   - 재시도 횟수 늘리기

2. **품질이 낮을 때**
   - 프롬프트 스타일을 'expert'로 변경
   - 더 큰 모델 사용
   - RAG 활성화

3. **속도가 느릴 때**
   - 통합형 모드 사용
   - max_new_tokens 줄이기
   - num_beams = 1로 설정